# Wikipedia Notable Life Expectancies
# [Notebook  14: Data Cleaning Part 13](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean13_thanak_2022_08_07.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### [Reading](), Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean12.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean12", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98056 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,1,0,0,0,0,0,0,1
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98054,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,1,0,0,1,0,0,0,2
98055,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [4]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98056 entries, 0 to 98055
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98056 non-null  object 
 1   name                       98056 non-null  object 
 2   info                       98056 non-null  object 
 3   link                       98056 non-null  object 
 4   num_references             98056 non-null  int64  
 5   year                       98056 non-null  int64  
 6   month                      98056 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     23 non-null     object 
 9   info_2                     98024 non-null  object 
 10  info_3                     48896 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by rebuilding `known_for_dict`.

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [5]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [6]:
# Code to check each value
roles_list.pop()

''

<IPython.core.display.Javascript object>

In [7]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "nazi" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [8]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

['nazi eugenicist', 'nazi']

<IPython.core.display.Javascript object>

In [9]:
# Example code to quick-check a specific entry
df[df["info_2"] == "nazi"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
5177,21,Hans Egon Holthusen,", 83, German nazi, writer and academic.",https://en.wikipedia.org/wiki/Hans_Egon_Holthusen,4,1997,January,None,None,nazi,writer and academic,None,None,None,None,None,None,None,None,83.0,None,Germany,None,None,1.609438,0,0,0,0,0,0,0,0,0,0,0,0,0
7609,15,Gunter d'Alquen,", 87, German nazi correspondent.",https://en.wikipedia.org/wiki/Gunter_d%27Alquen,6,1998,May,None,None,nazi,None,None,None,None,None,None,None,None,None,87.0,None,Germany,None,None,1.945910,0,0,0,0,0,1,0,0,0,0,0,0,1
11847,28,Heinrich Schmidt,", 88, German nazi physician.",https://en.wikipedia.org/wiki/Heinrich_Schmidt_(SS_doctor),5,2000,November,None,None,nazi,None,None,None,None,None,None,None,None,None,88.0,None,Germany,None,None,1.791759,1,0,0,0,0,0,0,0,0,0,0,0,1
12948,18,Karl Friedrich Titho,", 90, German nazi criminal.",https://en.wikipedia.org/wiki/Karl_Friedrich_Titho,5,2001,June,None,None,nazi,None,None,None,None,None,None,None,None,None,90.0,None,Germany,None,None,1.791759,0,0,0,0,0,0,0,0,0,1,0,0,1


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [10]:
# Creating lists for each category
politics_govt_law = [
    "advocate for raising awareness of epidermolysis bullosa",
    "advocate for the homeless and mentally ill",
    "CDC spokesperson and anti tobacco advocate",
    "advocate for women and transgender rights",
    "atheism and reproductive rights advocate",
    "advocate for the rights of prostitutes",
    "advocate for women and migrant workers",
    "cystic fibrosis assistance advocate",
    "councillor and community advocate",
    "breast cancer awareness advocate",
    "advocate of freedom of the press",
    "migrant workers' rights advocate",
    "and anti death penalty advocate",
    "and mental health care advocate",
    "disabled people rights advocate",
    "advocate for disability rights",
    "and indigenous rights advocate",
    "nuclear arms control advocate",
    "advocate for homeless rights",
    "transgender rights advocate",
    "consumer and women advocate",
    "endangered species advocate",
    "disability rights advocate",
    "prisoners' rights advocate",
    "indigenous rights advocate",
    "renewable energy advocate",
    "Rohingya rights advocate",
    "workers' rights advocate",
    "Tolowa cultural advocate",
    "consumer rights advocate",
    "women equality advocate",
    "social justice advocate",
    "cancer patient advocate",
    "women literacy advocate",
    "migrant worker advocate",
    "patient rights advocate",
    "and Men rights advocate",
    "animal rights advocate",
    "mental health advocate",
    "tribal rights advocate",
    "independence advocate",
    "advocate for the deaf",
    "anti smoking advocate",
    "euthanasia advocate",
    "insurance advocate",
    "advocate for Jews",
    "cycling advocate",
    "health advocate",
    "rights advocate",
    "autism advocate",
    "women advocate",
    "peace advocate",
    "LGBT advocate",
    "advocate",
    "chairwoman of the Nottawaseppi Huron Band of Potawatomi since",
    "first woman Treasurer of the",
    "trans woman pioneer",
    "Warumungu woman",
    "stateswoman",
    "acting Chief Justice of the Supreme Court of",  # before arts
    "nazi eugenicist",
    "nazi",
]

arts = [
    "traditional waka builder",
    "theatrical set builder",
    "shoemaker and bootmaker",
    "traditional cheesemaker",
    "guitar equipment maker",
    "waterfowl decoy maker",
    "and furniture maker",
    "MC and track maker",
    "paper doll maker",
    "organ maker and",
    "woodwind maker",
    "recorder maker",
    "bootmaker and",
    "bagpipe maker",
    "guitar maker",
    "cabinetmaker",
    "cheesemaker",
    "print maker",
    "knife maker",
    "movie maker",
    "screenmaker",
    "watchmaker",
    "kite maker",
    "glassmaker",
    "moviemaker",
    "sign maker",
    "shoemaker",
    "toy maker",
    "dollmaker",
    "trans woman and performer",
    "anchorwoman",
    "and acting coach",
    "acting coach",
    "acting",
]

sports = [
    "builder and team owner",
    "founder of Lola Cars",
    "sharpshooter",
    "racewalker",
    "Hall of Fame track and field coach",
    "track and field coach and",
    "track and field coach",
]
sciences = [
    "Hall of Fame NASCAR engine",
    "race car builder and",
    "yacht and boat builder",
    "framebuilder",
    "frame maker",
    "macrobiotic diet advocate",  # before politics_govt_law
    "privacy",
    "primatologist",
    "optometrist",
    "lichenologist",
]

business_farming = [
    "Chicago area home builder",
    "watch manufacturer",
    "importer",
    "wine maker",
    "gunmaker",
    "carmaker",
    "vegetarianism advocate",  # before politics_govt_law
    "co founder of Nike",
]
academia_humanities = ["polymath"]
law_enf_military_operator = [
    "chief bombmaker of Hamas",
    "chief executioner of Virginia",
    "only woman to serve in the Foreign Legion",
    "servicewoman during WWII",
    "former comfort woman",
    "servicewoman",
    'battalion commander known as "the executioner of Fort Zeelandia"',
    "former commander in chief of the Strategic Air Command",
    "former commander of the Corps of Gendarmerie of City",
    "warlord and commander of Federal Security Service",
    "divisional commander in the Waffen SS during WWII",
    "commander of the Waffen SS during Worls War II",
    "commander of the SAS during WWII",
    "commander of the Liberation Army",
    "commander in the Royal Air Force",
    "Revolutionary Guard commander",
    "former labor camp commander",
    "separatist field commander",
    "Provisional IRA commander",
    "DPR separatist commander",
    "Army brigade commander",
    "rebel field commander",
    "mujahideen commander",
    "air force commander",
    "WWII tank commander",
    "Navy SEAL commander",
    "al Qaeda commander",
    "guerilla commander",
    "militia commander",
    "Taliban commander",
    "U boat commander",
    "rebel commander",
    "field commander",
    "South commander",
    "army commander",
    "Lehi commander",
    "ISIL commander",
    "PIRA commander",
    "FARC commander",
    "IRA commander",
    "SAS commander",
    "commander",
    "field marshal and chief of Army Staff",
    "Army field marshal",
    "field marshal",
    "firefighter",
    "Māori navigator and",
]
spiritual = ["occultist  practitioner", "occultist", "occult", "spiritualist"]
social = [
    "youth movement founder",
    "literacy advocate",  # before politics_govt_law
]
crime = [
    "concentration camp",
]
event_record_other = [
    "homemaker",
    "domestic abuse survivor",
    "Guinness World Record holder for heaviest woman",
    "woman whose adoption led to ethnic riots in in",
    "brain dead woman kept alive to give birth",
    "woman believed to be world oldest mother",
    "woman with rare slow aging condition",
    "heaviest woman in the world",
    "tallest woman in the world",
    "transgender woman",
    "tattooed woman",
    "mauled woman",
]
other_species = []

<IPython.core.display.Javascript object>

In [11]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

In [12]:
# Hard-coding info_2 value for entry to categorize as business_farming
index = df[df["link"] == "https://en.wikipedia.org/wiki/Harry_Henshel"].index
df.loc[index, "info_2"] = "watch manufacturer"

# Hard-coding info_2 value for entry to categorize as crime
index = df[df["link"] == "https://en.wikipedia.org/wiki/Venero_Mangano"].index
df.loc[index, "info_2"] = "mafia"

<IPython.core.display.Javascript object>

In [13]:
# Dropping entry that points to event page rather than individual's page and resetting index
index = df[
    df["link"] == "https://en.wikipedia.org/wiki/Death_of_Paolo_Gislimberti"
].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [14]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "sports": sports,
    "politics_govt_law": politics_govt_law,
    "arts": arts,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [15]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 30s
Wall time: 1min 30s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
86423,2,Michael McKevitt,", 71, Irish republican, founder of Real IRA, cancer.",https://en.wikipedia.org/wiki/Michael_McKevitt,27,2021,January,None,None,,founder of Real IRA,cancer,None,None,None,None,None,None,None,71.0,None,Ireland,None,None,3.332205,0,0,0,0,0,0,0,0,1,0,0,0,1
67288,27,Jerry Butler,", 58, American pornographic actor, cancer.",https://en.wikipedia.org/wiki/Jerry_Butler_(actor),8,2018,January,None,None,,cancer,None,None,None,None,None,None,None,None,58.0,None,United States of America,None,None,2.197225,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [16]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 2677 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [17]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [18]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [19]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "ace" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [20]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [21]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "race"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [22]:
# Creating lists for each category
politics_govt_law = [
    "politologist",
    "banking regulator",
    "econo",
    "elder",
    "social reformer",
    "and member of n State Duma",
    "claimant to headship of the House of Bourbon Two Sicilies",
    "claimant to the headship of the House of Romanov",
    "labor mediator",
    "plaintiff in landmark 'palimony' lawsuit",
    "international law expert",
    "born constitutional law",
    "constitutional law",
    "international law",
    "Labour Member of Parliament for Dunfermline and West Fife",
    "Minister of Labour",
    "Labour",
    "public",
    "Economics",
    "co founder of Greenpeace",
    "peace",
]

arts = [
    "antique dealer and ologist",
    "horologist",
    "variety performer with his brother Bob as half of Bob and Alf Pearson",
    "hustler and nightclub performer",
    "San Francisco street performer",
    "circus performer and owner",
    "Villu Paatu performer",
    "vaudeville performer",
    "burlesque performer",
    "gamelan performer",
    "circus performer",
    "kabuki performer",
    "Kunqu performer",
    "drag performer",
    "performer",
    "website pioneer of ethnic media in",
    "media proprietor",
    "Cheyenne flutist",
    "concert flutist",
    "flutist",
    "newspaper proprietor",
    "newsreel narrator",
    "newspaper founder",
    "newspaper website",
    "newspaper pioneer",
    "newspaper owner",
    "news cameraman",
    "and newsreader",
    "newspaperman",
    "newsreader",
    "newspaper",
]
sports = [
    "figure skating competitor",
    "and figure skating coach",
    "figure skating trainer",
    "figure skating coach",
    "ice skating trainer",
    "speed skating coach",
    "race caller",
]
sciences = [
    "jet engine pioneer",
    "conchologist and malacologist",
    "taxonomist and carcinologist",
    "pulmonologist and internist",
    "cytologist and phycologist",
    "paediatric rheumatologist",
    "echinodermologist and",
    "vintner and",
    "pomologist and apple",
    "dendrochronologist",
    "seed technologist",
    "food technologist",
    "nanotechnologist",
    "biogerontologist",
    "anesthesiologist",
    "geomorphologist",
    "biotechnologist",
    "helminthologist",
    "phenomenologist",
    "parasitologist",
    "rheumatologist",
    "ophthamologist",
    "gerontologist",
    "pulmonologist",
    "carcinologist",
    "orchidologist",
    "methodologist",
    "suicidologist",
    "malacologist",
    "nematologist",
    "hepatologist",
    "technologist",
    "polemologist",
    "dendrologist",
    "speleologist",
    "limnologist",
    "hydrologist",
    "cosmologist",
    "phycologist",
    "metrologist",
    "leprologist",
    "andrologist",
    "petrologist",
    "audiologist",
    "sexologist",
    "ethologist",
    "oenologist",
    "pedologist",
    "metrician",
    "obstetrician",
    "futurist",
    "software developer",
    "opinion pollster",
    "pollster",
    "physical therapist",
    "family therapist",
    "physiotherapist",
    "sex therapist",
    "therapist",
    "spacesuit technician",
    "space",
]

business_farming = [
    "bookmaker",
    "vineyard owner and vintner",
    "Napa Valley vintner",
    "vintner",
    "duty free retailer",
    "clothing retailer",
    "luxury retailer",
    "retailer",
    "venture capitalist and",
    "venture capitalist",
    "shipowner",
    "pharmaceutical company founder",
    "pharmaceutical",
]
academia_humanities = [
    "museum specialist",
    "Slavist and Balkanologist",
    "Kremlinologist",
    "vexillologist",
    "theatrologist",
    "Assyriologist",
    "Hittitologist",
    "assyriologist",
    "hittitologist",
    "Tibetologist",
    "Albanologist",
    "egyptologist",
    "Japanologist",
    "papyrologist",
    "polemologist",
    "Buddhologist",
    "sumerologist",
    "tibetologist",
    "lexicologist",
    "Iranologist",
    "museologist",
    "patrologist",
    "mythologist",
    "semiologist",
    "Turkologist",
    "anthologist",
    "indologist",
    "logologist",
    "genealogist and heraldist",
    "heraldist",
    "Arabist",
    "mediaevalist",
    "Wikimedian",
    "phonetician",
]
law_enf_military_operator = [
    "cryptologist",
    "operative",
    "Ulster loyalist and UVF member",
    "Ulster loyalist",
    "loyalist",
    "warlord",
    "retired Navy Vice Admiral",
    "Navy Vice Admiral",
    "Vice Admiral",
    "brother in law of Osama bin Laden",
    "law enforcement official",
    "outlaw biker",
    "lawman",
    "member of the Resistance during WW II",
    "member of the Resistance during WWII",
    "a member of the Resistance",
    "member of the Resistance",
    "anti Castro mercenary",
    "mercenary",
    "bodyguard",
    "last surviving World War I fighter ace",
    "fighter ace during World War ||",
    "Air Force major and flying ace",
    "flying ace during World War I",
    "Army Air Forces fighter ace",
    "Army Air Force fighter ace",
    "Army Air Forces flying ace",
    "flying ace of the War",
    "Air Force flying ace",
    "WWII fighter ace",
    "WWII flying ace",
    "War flying ace",
    "Air Force ace",
    "fighter ace",
    "flying ace",
]
spiritual = [
    "demonologist",
    "spiritual guardian of Mount Merapi",
    "faith healer",
    "spiritual guru",
    "Grand Ayatollah",
]
social = []
crime = []
event_record_other = [
    "woman",
    "claimant to the title of world oldest person",
    "unverified claimant for world oldest person",
    "alien abduction claimant",
    "longevity claimant",
    "girl whose severe swimming pool injury led to tougher laws",
    "lawn mower traveler",
    "airplane passenger fatally shot by Air Marshals after allegedly claiming he had placed a bomb aboard",
    "face transplant recipient",
]
other_species = [
    "Hindu sacred bull",
    "oldest living polar bear",
    "grizzly bear",
    "polar bear",
    "tortoise claimant for world oldest animal",
    "Thoroughbred race horse and sire",
    "Thoroughbred race horse",
    "thoroughbred race horse",
    "race horse",
]

<IPython.core.display.Javascript object>

In [23]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [24]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [25]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 52s
Wall time: 1min 52s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
22169,10,Carlo Acutis,",15, beautified catholic teenager",https://en.wikipedia.org/wiki/Carlo_Acutis,29,2006,October,None,None,,None,None,None,None,None,None,None,None,None,15.0,None,None,None,None,3.401197,0,0,1,0,0,0,0,0,0,0,0,0,1
63598,15,Stan Kaluznick,", 85, Canadian football player .",https://en.wikipedia.org/wiki/Stan_Kaluznick,4,2017,May,Calgary Stampeders,None,,None,None,None,None,None,None,None,None,None,85.0,None,Canada,None,Calgary Stampeders,1.609438,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [26]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 2365 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [27]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [28]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [29]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "car dealer" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [30]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [31]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "milliner"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [32]:
# Creating lists for each category
politics_govt_law = [
    "regulator",
    "law",
    "constitutional",
    "hacker",
    "former press secretary for Richard Nixon during the Watergate Scandal",
    "secretary and mistress of Reichsführer SS Heinrich Himmler",
    "secretary who took Adolf Hitler last will and testament",
    "national security adviser and former foreign secretary",
    "Special Assistant and secretary to John F Kennedy",
    "private secretary to Queen Elizabeth II",
    "first secretary of the Leningrad obkom",
    "secretary of foreign affairs",
    "press secretary",
    "heir to the Sarawakan throne",
    "colonial official",
    "colonial governor",
    "anti colonialist",
    "colonial",
    "Trotskyist and one of the founders of the Revolutionary Socialist League",
    "Trotskyist",
    "international relations",
    "Foreign Service Officer and ambassador to and",
    "ambassador to the United Nations",
    "United Nations ambassador",
    "former ambassador to",
    "Arabian ambassador",
    "ambassador to the",
    "and ambassador",
    "ambassador to",
    "ambassador",
    "detainee in Guantanamo Bay Detention Camp",
    "Arabian Guantanamo Bay detainee",
    "former Guantanamo Bay detainee",
    "ex detainee",
    "detainee",
    "Liberal Democrat Member of Parliament",
    "Conservative Member of Parliament",
    "Member of Parliament",
]

arts = [
    "maker",
    "organ builder",
    "yurt builder",
    "glass engraver",
    "media",
    "opera",
    "fashion house owner",
    "fashion promoter",
    "fashion stylist",
    "fashion pioneer",
    "fashion",
    "ballet master and",
    "ballet master",
    "and ballet",
    "ballet",
    "Māori muralist",
    "muralist",
    "pioneer in visual effects and computer animation",
    "virtuoso harmonica player",
    "harmonica player",
    "SeaWorld trainer",
    "animal trainer",
    "litterateur",
    "pornographer",
    "R&B",
    "textile weaver and dyer",
    "Māori master weaver",
    "tapestry weaver",
    "Tlingit weaver",
    "carpet weaver",
    "basketweaver",
    "weaver",
    "punk impresario and club owner",
    "theatrical impresario",
    "nightclub impresario",
    "nightlife impresario",
    "cabaret impresario",
    "impresario",
    "wildlife cameraman",
    "Reuters cameraman",
    "cameraman",
    "sarod player",
    "hairstylist",
    "classic Broadway star and father of Bonnie Raitt",
    "action movie star",
    "reality TV star",
    "movie star",
    "co star of",
    "porn star",
    "bassoonist",
    "screenwriting guru",
    "screen",
    "milliner",
]
sports = [
    "strength coach",
    "Muay fighter",
    "MMA fighter",
    "Muay master and trainer",
    "harness racing trainer",
    "Thoroughbred trainer",
    "greyhound trainer",
    "fitness trainer",
    "fencing trainer",
    "rowing coxswain",
    "rowing champion",
    "former National Football League player",
    "ex National Football League player",
    "National Football League player",
    "ski mogul",
    "former Football League player and NHL referee",
    "former National Football League player",
    "ex National Football League player",
    "National Football League player",
    "Football League player",
    "professional basketball referee",
    "basketball referee and coach",
    "basketball referee",
    "Hall of Fame softball player",
    "softball player",
    "motor racing team owner and constructor",
    "drag racing and hot rod pioneer",
    "motor racing team owner",
    "auto racing team owner",
    "car racing team owner",
    "auto racing pioneer",
    "car racing promoter",
    "racing team owner",
    "racing crew chief",
    "mountain biker",
    "mountain guide",
    "basketball star and innovator",
    "Hall of Fame rodeo cowboy",
    "rodeo cowboy",
    "rodeo rider",
    "rodeo",
]
sciences = [
    "car builder",
    "and enologist",
    "anatomist",
    "Internet software pioneer and computer prodigy",
    "computer expert",
    "quantum physics",
    "physics",
    "elephant expert",
    "CERN secretary",
    "herbalist",
    "Scientist and Medical Researcher",
    "mathematical",
    "game developer",
    "clinician",
    "Nurse Corps",
    "hospital corpsman",
    "genealogist",
]

business_farming = [
    "oilman",
    "real estate tycoon",
    "timber tycoon",
    "tire tycoon",
    "cattle baron",
    "real estate mogul",
    "automotive dealer",
    "insurance broker",
    "clothier and",
    "clothier",
    "whisky distiller",
    "drink distiller",
    "distiller",
    "car dealer",
]
academia_humanities = [
    "heraldry",
    "Esperantist",
    "Germanist",
    "Hellenist",
    "ethnographer",
]
law_enf_military_operator = [
    "East border guard",
    "Waffen SS  guard",
    "security guard",
    "guard",
    "security detective",
    "founder of Al Qaeda",
    "Al Qaeda",
    "Field Marshal",
    "Tzotzil Zapatista rebel",
    "rebel",
    "al Qaeda fighter in Chechnya",
    "oil well fire fighter",
    "independence fighter",
    "WW Special Forces saboteur and trainer of Violette Szabó",
    "resistance trainer",
    "ISIS trainer",
    "sergeant and recipient of the Victoria Cross",
    "war hero and recipient of the Victoria Cross",
    "Gurkha and recipient of the Victoria Cross",
    "Pashtun recipient of the Victoria Cross",
    "recipient of the Victoria Cross",
    "Navy chief cryptologic technician",
    "Navy Commander in Chief",
    "Secretary of the Navy",
    "Director of the Navy",
    "Navy Master Diver",
    "Navy SEAL sniper",
    "head of the Navy",
    "Navy captain",
    "Navy SEAL",
    "Navy",
    "bibliographer",
    "intelligence official who was Deputy Director of the Central Intelligence Agency",
    "intelligence official in the Central Intelligence Agency",
    "former head of East y secret intelligence service",
    "army Green Beret and intelligence sergeant major",
    "intelligence chief of the National Authority",
    "Chief of intelligence of South",
    "head of intelligence services",
    "imagery intelligence analyst",
    "intelligence specialist",
    "intelligence official",
    "intelligence analyst",
    "intelligence chief",
    "Air Force navigator",
    "flight navigator",
    "navigator",
    "marshal of the air force",
    "Air Force marshal",
    "air vice marshal",
    "vice marshal",
    "marshal",
]
spiritual = [
    "Hindu reformer",
    "Hindu guru",
    "Hinduism",
    "Hindu",
    "fortune teller made famous in Bruce Springsteen",
    "fortune teller",
    "ecclesiastical",
    "grand ayatollah",
    "Shi'i ayatollah",
    "ayatollah",
    "and a founding father of the New Age movement",
    "yogi and guru",
    "yogi",
    "Chabad Lubavitch Rabbi and secretary to Menachem Schneerson",
    "Lakota medicine man",
    "Twelver Marja'",
    "Marja'",
    "New Testament",
]
social = []
crime = [
    "failed suicide bomber",
    "organized crime figure in Detroit",
    "organized crime figure",
    "organised crime figure",
    "crime figure",
]
event_record_other = ["homeless man"]
other_species = [
    "Humboldt penguin",
    "penguin",
    "foaled thoroughbred horse",
    "thoroughbred horse",
    "Standardbred harness racing stallion",
    "breeding stallion and racing trotter",
    "barrel racing horse",
    "racing thoroughbred",
    "dressage horse and sire",
    "dressage horse",
    "canine star of sit com Eddie",
    "Great Dane",
]

<IPython.core.display.Javascript object>

In [33]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [34]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [35]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 2min 8s
Wall time: 2min 8s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
24318,8,Stephen Fumio Hamao,", 77, Japanese Roman Catholic cardinal, former bishop of Yokohama, lung cancer.",https://en.wikipedia.org/wiki/Stephen_Fumio_Hamao,4,2007,November,None,None,,former bishop of,lung cancer,None,None,None,None,None,None,None,77.0,None,Japan,Italy,None,1.609438,0,0,1,0,0,0,0,0,0,0,0,0,1
23882,17,Carolyn Goodman,", 91, American psychologist and civil rights activist.",https://en.wikipedia.org/wiki/Carolyn_Goodman_(psychologist),5,2007,August,None,None,,None,None,None,None,None,None,None,None,None,91.0,None,United States of America,None,None,1.791759,1,0,0,0,0,0,0,0,1,0,0,0,2


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [36]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 2041 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [37]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [38]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [39]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "protest" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [40]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [41]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "protest"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [42]:
# Creating lists for each category
politics_govt_law = [
    "herald",
    "foreign policy analyst",
    "policy analyst",
    "food analyst",
    "baroness and",
    "baroness",
    "baronet",
    "baron",
    "transportation official",
    "transportation and agri",
    "Minister of Transport",
    "Supreme Court justice",
    "first female justice",
    "High Court justice",
    "associate justice",
    "justice",
    "secretary",
    "deputy tribal chief of the Nation",
    "traditional tribal chief",
    "Athabascan tribal chief",
    "Seminole tribal chief",
    "Mohegan tribal chief",
    "Wayana tribal chief",
    "tribal chief",
    "women rights pioneer",
    "activist",
    "implicated in the murder of Daniel Pearl",
    "industry communicator",
    "proliferation expert",
    "food security expert",
    "Minister of Communications",
    "Commissioner of Education",
    "Minister of Education",
    "Education Minister",
    "government adviser of Milton Obote and Idi Amin",
    "scientific adviser",
    "adviser",
    "libertarian",
    "tax protestor",
    "protestor",
    "butler and staffer",
    "butler",
    "finance official",
    "protest",
]

arts = [
    "ologist",
    "cultural analyst",
    "antiques expert",
    "rock concert promoter",
    "concert promoter",
    "landscape gardener",
    "gardener",
    "woodcarver",
    "Carna",
    "beer expert",
    "tailor",
    "blacksmith",
    "luthier who buildt archtop guitars",
    "luthier",
    "rock multi instrumentalist",
    "promoter of punk rock",
    "rock 'n' roll groupie",
    "rock concert promoter",
    "rock and roll",
    "promoter of LGBTQ cinema",
    "cinema  pioneer",
    "cinema",
    "festival organizer",
    "horn player",
    "photography",
    "dialect coach",
]
sports = [
    "eng",
    "archer",
    "bicycle",
    "basketball pioneer who popularized the sport in City",
    "and sport franchise owner",
    "extreme sport practitioner",
    "motor sport",
    "sporting",
    "alpinist",
    "world champion field handball team player",
    "handball and beach handball coach",
    "handball coach and player",
    "handball referee",
    "handball coach",
    "handballer",
    "motorcycling",
    "center on the University of Arizona women basketball team",
    "coach of Naval Academy men and women basketball teams",
    "One of the first women to swim the Channel",
    "MLB All Star catcher",
    "CFL coach",
    "freestyle motocross rider",
    "motocross champion",
    "motocross rider",
    "shogi player",
    "track coach",
    "VFL player",
    "wheelchair marathoner and paralympian",
    "paralympian",
    "professional bull rider",
    "backgammon player",
    "BMX rider",
    "coxswain",
]
sciences = [
    "computer",
    "ine builder",
    "systems analyst",
    "practitioner of traditional medicine",
    "alternative medicine practitioner",
    "hyperbaric medicine specialist",
    "forensic medicine",
    "clinical medicine",
    "social medicine",
    "medicine",
    "zookeeper",
    "biostatistics",
    "statistics",
    "automotive expert and",
    "automotive",
    "motoring",
    "anaesthetist",
    "nuclear",
    "bee expert",
    "rocket expert",
    "developer of the bulletproof Kevlar vest",
]

business_farming = [
    "home builder",
    "wallpaper heir and stepfather of Diana",
    "publishing heir and",
    "heir and",
    "heir",
    "casino owner",
    "arms dealer",
]
academia_humanities = [
    "Hispanist",
    "Mayanist",
    "pioneer of women studies",
]
law_enf_military_operator = [
    "munitions analyst",
    "seaman",
    "gendarme",
    "CIA official",
    "nuclear strategist",  # before sciences
]
spiritual = ["catechist", "Salvation Army General"]
social = [
    "cancer research fundraiser",
    "hospice fundraiser",
    "and fundraiser",
    "fundraiser",
    "co founder of the Findhorn Foundation",
]
crime = [
    "drug dealer",
    "Genovese family crime boss",
    "organized crime boss",
    "crime boss",
]
event_record_other = [
    "last surviving Kindertransport mother",
    "pensioner",
    "cattle grazier",
    "obese man",
    "primordial dwarf",
    "dwarf",
]
other_species = [
    "Asian elephant",
    "elephant",
    "Groundhog Day prognosticator",
    "world longest domestic cat",
    "pet cat of George W Bush",
    "Downing Street cat",
    "domestic cat in",
    "polydactyl cat",
    "Himalayan cat",
    "Ragdoll cat",
    "Library cat",
    "calico cat",
    "tabby cat",
    "obese cat",
    "stray cat",
    "victim",
    "Thoroughbred horse",
    "world champion bucking bull",
    "bucking bull",
    "show jumping horse",
    "eventing horse",
    "thoroughbred stallion",
    "thoroughbred",
]

<IPython.core.display.Javascript object>

In [43]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

In [44]:
# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Norm_Grabowski"].index
df.loc[index, "info_2"] = "engine builder"

# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Dimitris_Christoulas"].index
df.loc[index, "info_2"] = "activist"  # added to dict

# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Waldo_McBurney"].index
df.loc[index, "info_2"] = ""

# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Murder_of_Biswajit_Das"].index
df.loc[index, "info_2"] = "victim"  # added to dict

# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Eric_the_Actor"].index
df.loc[index, "info_2"] = "actor"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [45]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "sciences": sciences,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [46]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 35s
Wall time: 1min 35s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
3621,28,Dev Kant Baruah,", 81, Indian politician.",https://en.wikipedia.org/wiki/D._K._Barooah,3,1996,January,None,None,,None,None,None,None,None,None,None,None,None,81.0,None,India,None,None,1.386294,0,0,0,0,0,0,0,0,1,0,0,0,1
92915,22,Ulf Nilsson,", 73, Swedish children's author.",https://en.wikipedia.org/wiki/Ulf_Nilsson_(author),12,2021,September,None,None,,None,None,None,None,None,None,None,None,None,73.0,None,Sweden,None,None,2.564949,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [47]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 1838 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [48]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [49]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [50]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "rhinoceros" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [51]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [52]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "flight attendant"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [53]:
# Creating lists for each category
politics_govt_law = [
    "Surveyor of thepretender to the throne of the Kingdom of Araucania and Patagonia",
    "known for campaigns against fast food chains and food processors",
    "press official under Richard M Nixon and Gerald R Ford",
    "former Imperial Wizard of the Ku Klux Klan",
    "Guantanamo Bay detention camp prisoner",
    "Ku Klux Klan member and segregationist",
    "United Nations disarmament official",
    "Queen of as the wife of King Talal",
    "sister of King Abdullah of Arabia",
    "Queen consort of Frederick IX of",
    "Queen consort of King Zog I of",
    "queen consort of King Baudouin",
    "th Emperor King of the people",
    "servant to HM Queen Elizabeth",
    "wrongly imprisoned prisoner",
    "who prosecuted Jimmy Hoffa",
    "consort of King George VI",
    "foreign policy official",
    "sister of King Abdullah",
    "foreign policy expert",
    "Ku Klux Klan member",
    "exonerated prisoner",
    "juridical official",
    "prisoner in gulag",
    "the last Queen of",
    "health reformer",
    "King of Wallis",
    "state official",
    "communicator",
    "door to door",
    "Arabian King",
    "ex prisoner",
    "suffragist",
    "Queen of",
    "marxist",
    "King of",
    "women",
    "King",
    "egal",
]

arts = [
    "traditional Māori instrumentalist",
    "health and fitness pioneer",
    "bluegrass mandolin player",
    "silversmith and goldsmith",
    "multi instrumentalist",
    "burlesque stripteaser",
    "bluegrass mandolinist",
    "classical mandolinist",
    "silver and goldsmith",
    "Emmy Award winning",
    "burlesque stripper",
    "pageant contestant",
    "circus ringmaster",
    "euphonium player",
    "antiques dealer",
    "mandolin player",
    "instrumentalist",
    "uilleann piper",
    "pageant winner",
    "pageant queen",
    "watercolorist",
    "silversmith",
    "goldsmith",
    "sitarist",
    "stripper",
    "cantor",
    "rock",
]
sports = [
    "National Football League coach and a member of the Pro Football Hall of Fame",
    "Football League coach with the Toronto Argonauts and Ottawa Rough Riders",
    "former coach of the Toronto Rock of the National Lacrosse League",
    "Virgin Islander Baltimore Orioles coach",
    "National Rugby League player and coach",
    "sprint kayaker and Paralympic coach",
    "National Football League official",
    "Basketball Hall of Fame coach",
    "cross country running coach",
    "coach of the Boston Celtics",
    "basketball official scorer",
    "swimming and diving coach",
    "draughts player and coach",
    "yoga as exercise pioneer",
    "fencing master and coach",
    "coach for A&M basketball",
    "futsal player and coach",
    "rugbyplayer and coach",
    "basketball official",
    "rugby league coach",
    "hammer throw coach",
    "ski jumping coach",
    "equitation coach",
    "water polo coach",
    "trampoline coach",
    "high jump coach",
    "croquet player",
    "hurling coach",
    "fencing coach",
    "qigong master",
    "cycling coach",
    "skiing coach",
    "rugby coach",
    "yoga master",
    "rejoneador",
    "judo coach",
    "swim coach",
    "yoga guru",
    "angler",
    "coach",
]
sciences = [
    "holistic health practitioner and naturopath",
    "mental health professional",
    "health care consultant",
    "aquarium official",
    "fluid dynamicist",
    "climate analyst",
    "forestry expert",
    "Jungian analyst",
    "mineralogist",
    "orthopedist",
    "forestry",
]

business_farming = [
    "founder of Hardee fast food restaurant",
    "investment advisor",
    "fast food pioneer",
    "land developer",
    "internet",
    "salesman",
    "finance",
    "builder",
    "grocer",
]
academia_humanities = [
    "Queen Works of Art",
    "Queen Pictures",
    "conservator",
    "transport",
]
law_enf_military_operator = [
    "Army master sergeant and recipient of the Medal of Honor",
    "Army sergeant and a recipient of the Medal of Honor",
    "Army Special Forces staff sergeant during the War",
    "Army Air Forces bombardier aboard the atomic bomb",
    "former sheriff of San Francisco and Cleveland",
    "Federal Aviation Administration official",
    "Lieutenant General of the People Army of",
    "Army combat medic and an armed member of",
    "Head of the Secret Intelligence Service",
    "head of the Secret Intelligence Service",
    "Central Intelligence Agency official",
    "Director of the Intelligence Bureau",
    "Major General in the Army Reserves",
    "Federal Judicial Police official",
    "chief air accident investigator",
    "Army command sergeant major",
    "sheriff of Dallas County",
    "Army Private First Class",
    "Army Lieutenant General",
    "ninth chief of the Army",
    "NCO and prisoner of war",
    "retired Army lieutenant",
    "Admiral of the Fleet",
    "private investigator",
    "Federal investigator",
    "Army first Sergeant",
    "Army Sergeant Major",
    "Officer in the Army",
    "technical sergeant",
    "deputy sheriff and",
    "fire investigator",
    "security official",
    "security expert",
    "Coast Guardsman",
    "prisoner of war",
    "Army brigadier",
    "county sheriff",
    "bomber crewman",
    "Army Sergeant",
    "Army corporal",
    "Rear Admiral",
    "investigator",
    "Army General",
    "Army Officer",
    "Army major",
    "serviceman",
    "paramedic",
    "maritime",
    "sheriff",
    "Admiral",
]
spiritual = [
    "founder of The Jesus Army",
    "evangelical Old Testament",
    "Sōtō Zen master",
    "evangelical",
    "Zen master",
    "religious",
    "of Islam",
    "mystic",
]
social = ["co creator of Trick or Treat for UNICEF"]
crime = ["waitress who converted to Islam", "serial burglar", "felon"]
event_record_other = [
    "Shortest adult human",
    "flight attendant",
    "health worker",
    "WWII prisoner",
]
other_species = [
    "trained chimpanzee believed to be first non human to acquire human language",
    "Baringo giraffe at the Santa Barbara Zoo with neck deformity",
    "and last surviving female northern white rhinoceros",
    "northern white rhinoceros",
    "giant panda",
    "chimpanzee",
    "rhinoceros",
    "amnesiac",
    "giraffe",
    "panda",
]

<IPython.core.display.Javascript object>

In [54]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

In [55]:
# Hard coding info_2 value for entries to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Lykourgos_Angelopoulos"].index
df.loc[index, "info_2"] = "religious cantor"  # religious added to dict

index = df[df["link"] == "https://en.wikipedia.org/wiki/Esther_Ghan_Firestone"].index
df.loc[index, "info_2"] = "religious cantor"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [56]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [57]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 45s
Wall time: 1min 45s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
95818,2,Ezio Frigerio,", 91, Italian costume designer and art director .",https://en.wikipedia.org/wiki/Ezio_Frigerio,3,2022,February,,None,,None,None,None,None,None,None,None,None,None,91.0,None,Italy,None,,1.386294,0,0,0,0,0,1,0,0,0,0,0,0,1
97847,24,Alfred Grishin,", 81, Russian boxing coach.",https://en.wikipedia.org/wiki/Alfred_Grishin,6,2022,May,None,None,,None,None,None,None,None,None,None,None,None,81.0,None,Russia,None,None,1.945910,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [58]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 1617 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [59]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [60]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [61]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "Hall of Fame" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [62]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [63]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "Hall of Fame"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [64]:
# Creating lists for each category
politics_govt_law = [
    "emir of ISIL in",
    "emir of",
    "emir",
    "Emir of Agaie",
    "Emir",
    "opium",
    "drug counselor",
    "labor organizer",
    "Associate Justice of the Supreme Court of the",
    "th Chief Justice of the Supreme Court",
    "Chief Justice of the Supreme Court of",
    "Chief Justice of the High Court of",
    "Chief Justice of the Supreme Court",
    "Justice of the Supreme Court",
    "Jurist and Chief Justice of",
    "Chief Justice from June",
    "Lord Chief Justice of",
    "Minister of Justice",
    "Chief Justice of",
    "Justice Minister",
    "Chief Justice",
    "Surveyor of the",
    "Maryland State Senator since",
    "Senator from Wyoming since",
    "Senator for Missouri",
    "State Senator",
    "Senator from",
    "Senator",
    "Union Gulag inmate",
    "sultan of the Aussa Sultanate",
    "foreign policy consultant",
    "Assistant Majority Leader in the Illinois House of Representatives",
    "member of the House of Representatives from Virginia since",
    "member of the House of Representatives from Indiana since",
    "member of the House of Representatives from Ohio since",
    "member of the House of Representatives from Illinois",
    "member of the House of Representatives from Nebraska",
    "member of the House of Representatives from New",
    "member of the Alabama House of Representatives",
    "member of the House of Representatives from",
    "member of the House of Representatives for",
    "Speaker of the House of Representatives",
    "member of the House of Representatives",
    "former Democratic Representative from",
    "Representative from Ohio since",
    "Representative from Washington",
    "Representative from Tennessee",
    "Democratic Representative for",
    "Representative from Michigan",
    "Representative for Wisconsin",
    "Representative from Maryland",
    "Representative for Oklahoma",
    "Democratic Representative",
    "Representative from since",
    "Representative from",
    "Representative for",
    "official",
    "governor of the Gila River Community",
    "governor of Bermuda from to",
    "Central Bank governor",
    "provincial governor",
    "governor",
    "black separatist",
    "pro separatist",
    "separatist",
]

arts = [
    "analyst",
    "adult",
    "Academy Award winning",
    "Bodil Award winning",
    "wargamer",
    "co founder of Filmation animation studios",
    "co creator of",
    "perfumer",
    "woodworker",
    "interior design consultant",
    "consultancy",
    "game show contestant",
]
sports = [
    "three cushion billiards player",
    "three cushion billiard player",
    "champion billiards player",
    "pocket billiards player",
    "billiards player",
    "goalkeeper",
    "cycling",
    "Lions rugby league captain",
    "rugby league referee",
    "rugby",
    "camogie player",
]
sciences = [
    "physical",
    "agriculturalist and recipient of the World Food Prize",
    "agriculturalist",
    "hydrographer",
    "Packard",
    "co founder of ECD Ovonics",
    "sociology and",
    "sociology",
    "emergency care",
    "quality consultant",
    "health",
    "pathology",
]

business_farming = [
    "co founder of Snapple Beverage Corporation",
    "co founder of Häagen Dazs ice cream",
    "co founder of Gaydar dating website",
    "co founder of E & J Gallo Winery",
    "co founder of Hewlett",
    "insurance consultant",
]
academia_humanities = ["co founder of the River and Rowing Museum", "anthropology"]
law_enf_military_operator = [
    "recipient of Righteous Among the Nations award",
    "Holocaust heroine and award winner",
    "war hero  Metropolitan",
    "war hero and",
    "airline stewardess",
    "warlady",
    "war heroine",
    "combat medic",
    "combat",
    "Scout and recipient of the Medal of Honor",
    "museum board member",
    "museum owner",
    "Air Force loadmaster and recipient of the Medal of Honor",
    "Commander of the National Air Force",
    "Chief of Staff of the Air Force",
    "General in the Air Force",
    "Air Force Commander",
    "Air Force Colonel",
    "Air Force",
]
spiritual = [
    "Old Testament",
    "second counselor in the First Presidency of the LDS Church",
    "Cardinal Deacon of San Francesco Saverio alla Garbatella",
    "Cardinal for San Pietro in Vincoli",
    "Cardinal",
    "motivational consultant",
    "Mormon",
]
social = [
    "substance abuse counselor",
    "family counselor",
    "drug counselor",
    "co founder of Habitat for Humanity International",
    "co founder of Twin Oaks Community",
    "professional Scouter for the Boy Scouts of",
    "oldest Girl Scout",
    "Scouter",
    "philanthropy consultant",
]
crime = [
    "death row inmate",
    "master counterfeiter",
    "counterfeiter",
    "vigilante",
    "polygamist",
]
event_record_other = []
other_species = [
    "war canine",
]

<IPython.core.display.Javascript object>

In [65]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [66]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [67]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 15s
Wall time: 1min 15s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
23396,25,Charles Nelson Reilly,", 76, American Tony-winning actor and panelist, complications from pneumonia.",https://en.wikipedia.org/wiki/Charles_Nelson_Reilly,24,2007,May,None,None,,complications from pneumonia,None,None,None,None,None,None,None,None,76.0,None,United States of America,None,None,3.218876,0,0,0,0,0,1,0,0,0,0,0,0,1
74192,10,Drupad Borgohain,", 77, Indian politician, MP , kidney disease.",https://en.wikipedia.org/wiki/Drupad_Borgohain,17,2019,April,,None,,MP,kidney disease,None,None,None,None,None,None,None,77.0,None,India,None,1998 2004,2.890372,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [68]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 1479 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [69]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [70]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [71]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "premier" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [72]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [73]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "premier"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [74]:
# Creating lists for each category
politics_govt_law = [
    "former Deputy Chief Minister of Maharashtra",
    "Chief Minister of Arunachal Pradesh",
    "stenographer to Winston Churchill",
    "former Chief Minister of Haryana",
    "who was first mayor of ancestry",
    "former premier of New Brunswick",
    "long serving mayor of Rosemont",
    "Minister for Foreign Affairs",
    "Deputy Premier of New South",
    "Federal Appeals Court Judge",
    "District and Circuit Judge",
    "mayor of Ocean Breeze Park",
    "Minister for Shipping and",
    "Foreign Affairs Minister",
    "Chief Minister of Delhi",
    "mayor elect of Westlake",
    "mayor of Atlantic City",
    "mayor of Nefteyugansk",
    "Public Works Minister",
    "Premier of East Sepik",
    "mayor of Vladikavkaz",
    "Minister of Industry",
    "member of Parliament",
    "mayor of La Gacilly",
    "nd mayor of Atlanta",
    "Minister for Sport",
    "Minister of Power",
    "mayor of Nagasaki",
    "county councillor",
    "Interior Minister",
    "Parliament member",
    "Cabinet Minister",
    "Foreign Minister",
    "boy demonstrator",
    "mayor of Buffalo",
    "mayor of Raleigh",
    "mayor of Dallas",
    "Chief Minister",
    "District Judge",
    "mayor of Flint",
    "Circuit Judge",
    "Vice Premier",
    "Premier of",
    "kidnapping",
    "prisoner",
    "mayor of",
    "premier",
    "inmate",
    "miner",
    "mayor",
]
arts = [
    "counter cultural figure",
    "counterculture figure",
    "counterculture",
    "country and",
    "comedy club owner",
    "club owner",
    "vedette",
    "B boy",
    "wilderness",
    "co founder of",
    "bagpiper",
    "bookbinder and craftsman",
    "craftsman",
    "wood carver",
    "carver",
    "timpanist",
    "hymnist",
    "script supervisor",
    "lithographer",
    "saxophone player",
    "muse",
    "food",
    "oud player",
    "conga player",
    "pioneering record promoter",
    "record company founder",
    "recording studio owner",
    "record label founder",
    "record label owner",
    "record store owner",
    "sound recording",
    "record promoter",
    "record dealer",
    "record",
    "steel guitar player for Vince Gill and Conway Twitty",
    "classical slide guitar player",
    "steel guitar player",
    "bass guitar player",
    "classical guitar",
    "guitar player",
    "guitar",
    "comedienne",
    "tuba player",
    "founder of Judges Guild",
    "boogie woogie",
    "santoor player",
    "winner of Nobel Prize for Literature",
]
arts = sorted(
    list(set(arts)), key=lambda x: len(x), reverse=True
)  # Sorting by decreasing length and removing duplicates
sports = [
    "member of the National Cowgirl Museum and Hall of Fame",
    "National Basketball Association Hall of Famer",
    "Hall of Fame show jumping rider",
    "Hall of Fame NASCAR team owner",
    "NASCAR Hall of Fame team owner",
    "Football Hall of Fame member",
    "Hall of Fame horsebreeder",
    "Pro Rodeo Hall of Famer",
    "cheerleading fan of the Dallas Cowboys",
    "survivalist",
    "multi discipline pistol shooter",
    "Paralympic shooter",
    "skeet shooter",
    "rifle shooter",
    "trap shooter",
    "futsal player",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)
sciences = [
    "probabilist",
    "forensic laboratory technician",
    "forensic expert",
    "forensic",
    "IT expert",
    "wildlife expert",
    "orthodontist",
    "pioneer in the field of magnetic tape sound",
    "Computers and Information at the University of Cambridge",
    "Computer Science",
    "biology",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)
business_farming = [
    "developer of a country club that did not admit black members",
    "accountancy",
    "accounting",
    "sex club owner",
    "multi millionaire",
    "multimillionaire",
    "millionaire",
    "ship owner",
    "founding head of Air Tahiti Nui",
    "CEO of Northwest Airlines",
    "accounting",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "analytic",
    "bibliophile",
    "semiotician",
    "consultant",
    "historical",
    "toponymist",
    "historiograph",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "encounter specialist with Mumbai Police",
    "war",
    "schoolboy",
    "survivor of Bataan Death March",
    "former KGB chief",
    "head of the KGB",
    "code breaker",
    "Air Vice Marshal",
    "RAAF Air Marshal",
    "Air Marshal",
    "hero of Liberation War",
    "first Minister of Defense",
    "Minister for Defence",
    "Minister of Defence",
    "Defense Minister",
    "chief of staff of the Provisional IRA",
    "navy chief of staff",
    "chief of staff",
    "member of the Tuskegee Airmen",
    "Chief of the Air Staff",
    "Tuskegee Airman",
    "informant for the FBI",
    "informant",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "Chief Rabbi of and of",
    "Chief Rabbi",
    "Rabbi",
    "salvationist",
    "Bishop of the Episcopal Diocese of Northern Michigan",
    "th Bishop of Massachusetts in The Episcopal Church",
    "Episcopal Bishop of West Tennessee",
    "Episcopal Bishop of Northwestern",
    "Auxiliary Bishop of Cincinnati",
    "Anglican Bishop of the Arctic",
    "Bishop of Mashonaland from to",
    "Bishop of Knaresborough",
    "Church of Bishop of",
    "Bishop of Malolos",
    "Bishop of Dallas",
    "Bishop of Oxford",
    "Anglican Bishop",
    "Bishop of Ripon",
    "Bishop of Selby",
    "Bishop of Como",
    "Bishop",
    "Saivite guru",
    "guru",
    "Executive Minister of the Iglesia ni Cristo",
    "oldest member of the Quorum of the Twelve Apostles in the history of The Church of Jesus Christ of Latter day Saints",
    "member of the Quorum of the Twelve Apostles of The Church of Jesus Christ of Latter day Saints",
    "patriarch emeritus of The Church of Jesus Christ of Latter day Saints",
    "apostle of The Church of Jesus Christ of Latter day Saints",
    "Elder of The Church of Jesus Christ of Latter day Saints",
    "Secretary General of the General Synod of the Church of",
    "Orthodox patriarch of the Orthodox Church of Antioch",
    "Supreme Primate of the Malankara Orthodox Church",
    "Pope of the Coptic Orthodox Church of Alexandria",
    "Metropolitan of Malankara Orthodox Church",
    "patriarch of the Orthodox Tewahedo Church",
    "primate of the Malankara Orthodox Church",
    "Protopresbyter of the Orthodox Church",
    "primate of the Orthodox Church since",
    "th Patriarch of the Orthodox Church",
    "Patriarch of the Orthodox Church",
    "Orthodox Church hierarch",
    "primate of the Church of",
    "Church of Bishop of",
    "LDS Church",
    "Wiccan",
    "spiritual healer",
    "spiritual figure",
    "spiritual",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
crime = [
    "bank robber and prison escapee of the s",
    "bank robber and con man",
    "bank robber",
    "reputed mafia enforcer",
    "mafia boss",
    "mafia",
    "suicide bomber and assassin of Beant Singh",
    "suicide bomber",
    "hotel bomber",
    "bomber",
    "'Ndrangheta boss",
    "school shooter",
    "alleged Mafia boss of Pittsburgh",
    "member of the Mafia",
    "former Mafia boss",
    "Mafia underboss",
    "Mafia hit man",
    "Mafia member",
    "Mafia boss",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "survivor of Hiroshima and Nagasaki atomic bombings",
    "survivor of the Sant'Anna di Stazzema massacre",
    "survivor of the sinking of RMS",
    "oldest living survivor of the",
    "last survivor of the sinking",
    "survivor of the Explosion",
    "Sinti Porajmos survivor",
    "survivor of Auschwitz",
    "shipwreck survivor",
    "abortion survivor",
    "gunshot survivor",
    "persistent vegetative state patient",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "cat",
]

<IPython.core.display.Javascript object>

In [75]:
# Dropping entry with link that points to event page rather than individual page
index = df[df["link"] == "https://en.wikipedia.org/wiki/Barbara_McDermott"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

# Dropping entry with link that points to event page rather than individual page
index = df[
    df["link"]
    == "https://en.wikipedia.org/wiki/2005_French_riots#Murders_of_Jean-Claude_Irvoas_and_Jean-Jacques_Le_Chenadec"
].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

# Dropping entry with link that points to event page rather than individual page
index = df[
    df["link"] == "https://en.wikipedia.org/wiki/Permanente_Quarry#2011_shooting"
].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

# Dropping entry with link that points to event page rather than individual page
index = df[df["link"] == "https://en.wikipedia.org/wiki/Glen_Doherty"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [76]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [77]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 2min
Wall time: 2min


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
71176,10,Tex Winter,", 96, American Hall of Fame basketball coach , innovator of the triangle offense.",https://en.wikipedia.org/wiki/Tex_Winter,16,2018,October,"Kansas State Wildcats, Chicago Bulls",None,,innovator of the triangle offense,None,None,None,None,None,None,None,None,96.0,None,United States of America,None,"Kansas State Wildcats, Chicago Bulls",2.833213,0,0,0,0,0,0,1,0,0,0,0,0,1
96039,12,Frank Beckmann,", 72, German-born American broadcaster , complications from vascular dementia.",https://en.wikipedia.org/wiki/Frank_Beckmann,15,2022,February,WJR,None,,complications from vascular dementia,None,None,None,None,None,None,None,None,72.0,None,Germany,United States of America,WJR,2.772589,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [78]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 1223 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [79]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [80]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [81]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "Black Panther" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [82]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [83]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "Sultan of Sokoto"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [84]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "sultan",
    "count",
    "Legislative Council member and chair of the DAB",
    "member of the Legislative Assembly of Alberta",
    "first female Lord Mayor of",
    "Lord Mayor of Birmingham",
    "first Mayor of Atlanta",
    "Lord Mayor of Brisbane",
    "Mayor of Pittsburgh",
    "Mayor of Auckland",
    "Mayor of Edmonton",
    "rd Mayor of City",
    "th Mayor of City",
    "Lord Mayor of",
    "Mayor of",
    "Chief of Staff and Treasury Secretary during the Reagan administration",
    "Secretary General of Karen National Union",
    "Secretary of the Interior since",
    "Assistant Secretary of Health",
    "Secretary of Commerce",
    "Secretary of Interior",
    "Secretary of State",
    "Finance Secretary",
    "Press Secretary",
    "Conservative member of the House of Lords",
    "crossbench member of the House of Lords",
    "Lord Lieutenant of Lincolnshire since",
    "Lord Lieutenant of Buckinghamshire",
    "Lord Lieutenant of Merseyside",
    "Member of the House of Lords",
    "first female Lord Mayor of",
    "Lord Provost of Aberdeen",
    "Lord Mayor of Birmingham",
    "Lord Mayor of Brisbane",
    "Lord Mayor of",
    "Law Lord",
    "founder of VHP",
    "Attorney General of Alabama",
    "Attorney General of",
    "Assistant Attorney",
    "member of the North Carolina General Assembly",
    "member of the Legislative Assembly of Alberta",
    "congressman from Tennessee and",
    "first openly gay congressman",
    "congressman from Indiana",
    "congressman from Idaho",
    "fourth and last wife of Sir Sultan Muhammad Shah",
    "Sultan of Sokoto",
    "Black Panther member",
    "Black Panther",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "Hall of Fame",
    "player of cello and viola da gamba",
    "flute and tin whistle player",
    "classical surbahar player",
    "bass and trombone player",
    "Creole fiddle player",
    "shakuhachi player",
    "didgeridoo player",
    "harmonium player",
    "gayageum player",
    "tárogató player",
    "bouzouki player",
    "requinto player",
    "clarinet player",
    "sarangi player",
    "ukulele player",
    "fiddle player",
    "cuatro player",
    "yidaki player",
    "sitar player",
    "gyile player",
    "veena player",
    "brass player",
    "gusle player",
    "piano player",
    "flute player",
    "pipa player",
    "dhol player",
    "bureau chief in Washington for the",
    "owner of Drury Lane Theatre",
    "owner of Fy Records",
    "bookshop owner",
    "boutique owner",
    "press owner",
    "bar owner",
    "bookstore proprietor",
    "audiobook narrator",
    "bookshop owner",
    "book reviewer",
    "bookbinder",
    "book",
    "controller of BBC Radio and The Proms",
    "controller of the BBC in Northern",
    "creator of Dudley Do Right and The Munsters",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "former MLB player for the Brooklyn Dodgers",
    "National Basketball Association player",
    "MLB player with the Boston Braves",
    "basketball and streetball player",
    "professional video game player",
    "Magic: The Gathering player",
    "shinty player and referee",
    "Paralympic boccia player",
    "Rugby Union player and",
    "world champion player",
    "racquetball player",
    "Euroleague player",
    "checkers player",
    "goalball player",
    "pétanque player",
    "kabaddi player",
    "pelota player",
    "bandy player",
    "ballplayer",
    "ice player",
    "NHL player",
    "go player",
    "and the Columbus Blue Jackets",
    "former owner of the Philadelphia ers who signed Julius Erving",
    "primary owner of the Cincinnati Reds",
    "owner of the NHL Chicago Blackhawks",
    "majority owner of St Louis Rams",
    "owner of Kansas City Chiefs",
    "owner of the Boston Red Sox",
    "Formula One team owner",
    "owner of hot rod shop",
    "stock car team owner",
    "owner of the Jets",
    "basketball owner",
    "Giants co owner",
    "CEO of the International Rugby Board",
    "CEO of NASCAR",
    "NHL goaltender and Vezina Trophy winner",
    "owner of the NHL Chicago Blackhawks",
    "NHL player",
    "promoter of Lucha Libre AAA World Wide",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "recipient of the Nobel Prize in Physiology or Medicine",
    "who created APT and led MIT CAD project",
    "of Science and Technology",
    "disk drive pioneer",
    "death from a fall off a bridge",
    "founder of freenode",
    "creator of Snobol and Icon programming languages",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "owner of Worthington Industries",
    "owner of Taillevent restaurant",
    "owner of Ameriquest Mortgage",
    "construction company owner",
    "steel company owner",
    "coffee house owner",
    "theme park owner",
    "gun shop owner",
    "company owner",
    "brothel owner",
    "airline owner",
    "estate owner",
    "zoo owner",
    "mine owner",
    "bar owner",
    "founder and CEO of Cubic Corporation",
    "founder and former CEO of ARCO",
    "CEO of Manufacturers Hanover",
    "CEO of The Hertz Corporation",
    "CEO of Xerox Corporation",
    "CEO of Jaguar Land Rover",
    "CEO of Sigma TV",
    "CEO of Chemoil",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "owner of the Military Vehicle Technology Foundation",
    "traditional owner",
    "founder of the SAIS Studies program",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "recipient of the George Cross and George Medal",
    "recipient of the Distinguished Conduct Medal",
    "recipient of the George Cross",
    "Victoria Cross recipient",
    "Military Medal recipient",
    "George Cross recipient",
    "Secretary of Defense under Reagan",
    "Watergate scandal principal",
    "Chief Constable of Greater Manchester Police",
    "Chief Constable for West Yorkshire",
    "Woman Police Constable",
    "small arms expert",
    "alleged Holocaust perpetrator",
    "last surviving Comanche code talker",
    "captain of the USS",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "and  and daughter of General Secretary Leonid Brezhnev",
    "founder of the Emmaüs movement",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["perpetrator", "stalker", "who hijacked a plane to"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "survivor",
    "boy",
    "oldest man in the at the time of his death",
    "who was the recognised world oldest person",
    "oldest living person of a possession",
    "and Manitoban oldest verified person",
    "oldest recognized person in the",
    "oldest living man",
    "oldest man",
    "owner of the Amityville Horror house",
    "barbecue stand owner",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "tortoise believed to be the third oldest animal in the world and allegedly owned by Charles Darwin",
    "Jack Russell terrier",
    "dachshund terrier",
    "terrier",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [85]:
# Dropping entry with link that points to husband's page
index = df[df["link"] == "https://en.wikipedia.org/wiki/Walter_Gretzky"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

# Hard-coding cause_of_death for entry with value in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Alan_MacDiarmid"].index
df.loc[index, "cause_of_death"] = "fall off bridge"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [86]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [87]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 38s
Wall time: 1min 38s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
25533,5,Hugh Bradner,", 92, American scientist credited with inventing the wetsuit, complications of pneumonia.",https://en.wikipedia.org/wiki/Hugh_Bradner,7,2008,May,None,None,,complications of pneumonia,None,None,None,None,None,None,None,None,92.0,None,United States of America,None,None,2.079442,1,0,0,0,0,0,0,0,0,0,0,0,1
80966,17,Georges Cukierman,", 93, French resistance member and communist activist.",https://en.wikipedia.org/wiki/Georges_Cukierman,5,2020,April,None,None,,None,None,None,None,None,None,None,None,None,93.0,None,France,None,None,1.791759,0,0,0,0,0,0,0,1,1,0,0,0,2


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [88]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 1043 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [89]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [90]:
# Code to check each value
roles_list.pop()

''

<IPython.core.display.Javascript object>

In [91]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "Prince" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [92]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

['Prince and Grand Master of the Sovereign Military Order of',
 'Princess and daughter of the Shah of',
 'reigning Prince of since',
 'former Princess of',
 'Prince']

<IPython.core.display.Javascript object>

In [93]:
# Example code to quick-check a specific entry
df[df["info_2"] == "advisor to Henry Kis"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [94]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Law and Dean of Law School",
    "Law School",
    "expert on Latin politics",
    "drug control expert",
    "agricultural expert",
    "welfare expert",
    "Head of the House of Hohenzollern Sigmaringen",
    "member of the House of Bourbon Two Sicilies",
    "member of the House of Hohenzollern",
    "last member of the House of Romanov",
    "Speaker of the House of Commons",
    "member of the House of Commons",
    "member of House of Commons",
    "hereditary chief of the Golden Hill Paugussett Nation since",
    "th chief of the Clan MacLeod",
    "Temagami First Nation chief",
    "traditional chief",
    "Neskonlith chief",
    "Crow Creek chief",
    "Blackfeet chief",
    "Catawba chief",
    "Māori chief",
    "at the College of Arms in",
    "member of the State Legislature since",
    "Business School",
    "queen consort of as wife of Shah Mohammad Reza Pahlavi",
    "rain queen of the Balobedu people of",
    "Māori queen",
    "Tuʻi Pelehake",
    "Ambassador to the Holy See",
    "Ambassador to",
    "Princess and daughter of the Shah of",
    "reigning Prince of since",
    "former Princess of",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "expert on the pipe organ",
    "gardening expert",
    "origami expert",
    "culinary",
    "flavorist who created the coating for Cap'n Crunch",
    "Playmate of the Year",
    "founder of Portmeirion Pottery",
    "talk",
    "pop queen",
    "Dean of Hypnotists",
    "late night broadcast pioneer",
    "reality series cast member",
    "broadcasting pioneer",
    "cast member of",
    "quilting",
    "Drummer for The Beau Brummels and Harpers Bizarre",
    "founder of Sonnabend Gallery",
    "creator of the Howdy Doody puppet",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "player",
    "survival expert",
    "five time world bridge champion",
    "contract bridge champion",
    "contract bridge",
    "dual medalist in slalom",
    "and national team captain",
    "NASCAR crew chief",
    "National Boxing Association middleweight champion",
    "National Football League header with the Pittsburgh Steelers and the Tampa Bay Buccaneers",
    "former defensive back for the Football Leagues Saskatchewan Roughriders",
    "commissioner of the National Football League",
    "Commissioner of the Football League",
    "casteller",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "co developer of the Billings ovulation method",
    "infectious disease expert credited with the HPV and rotavirus vaccines",
    "expert on survey methodology",
    "Mac OS X Server expert",
    "scientific expert",
    "acoustics expert",
    "fisheries expert",
    "snake expert",
    "shoe expert",
    "periodontist",
    "Nobel Prize in Medicine",
    "Medicine",
    "Sciences",
    "Microsoft pioneer",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "oil and gas expert",
    "wine expert",
    "last original tenant of the Empire State Building",
    "second richest man of and founder of Takefuji Corporation",
    "founder of Bob Evans Restaurants",
    "founder of Crazy John mobile phone retail chain",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "founder of Tennessee Temple University",
    "world renowned literacy expert",
    "expert on porcelain",
    "expert on history",
    "Esperanto expert",
    "Ferrari expert",
    "long serving Officer of Arms",
    "former Provost of Oriel College",
    "principal of Wolfson College",
    "fifth dean of the",
    "dean of the Yale University School of",
    "University dean of Arts and",
    "fifth dean of the Business School",
    "dean of the Academy",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "security and defence expert",
    "bomb disposal expert",
    "fingerprint expert",
    "polygraph expert",
    "additional commissioner of Mumbai Police",
    "Federal Police assistant commissioner",
    "first City Police Commissioner",
    "with Greater Manchester Police",
    "National Police Commissioner",
    "captain of the Hokulea first voyage from Hawaii to Tahiti",
    "Antarctic research ship captain",
    "captain and Hero of the Union",
    "captain of the SS",
    "airline captain",
    "SS captain and",
    "captain of",
    "chief of the Mumbai Anti Terrorist Squad",
    "secret service chief",
    "Secret Service chief",
    "air force chief",
    "defence chief",
    "militia chief",
    "army chief",
    "navy chief",
    "fire chief",
    "Former Chief Military",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "Dean of York from",
    "Prince and Grand Master of the Sovereign Military Order of",
    "founding abbot of Monks of New Skete",
    "abbot",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "executed for the rape and murder of two teenage girls in",
    "who arranged the murder of his mistress",
    "domestic worker executed in for murder",
    "executed for a double murder",
    "executed for murder in",
    "executed for the rape and murder of two teenage girls in",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "key witness in Emmett Till murder trial",
    "brain injured girl claimed to have performed miracles",
    "hermit",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "orca at SeaWorld San Antonio",
    "orca of and fame",
    "held orca",
    "orca",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [95]:
# Dropping entry with link that points to a list of individuals rather than one individual
index = df[df["link"] == "https://en.wikipedia.org/wiki/Camille_Loiseau"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [96]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [97]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 11s
Wall time: 1min 11s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
13981,19,Arkie Whiteley,", 37, Australian actress , adrenal cancer.",https://en.wikipedia.org/wiki/Arkie_Whiteley,7,2001,December,", ,",None,,adrenal cancer,None,None,None,None,None,None,None,None,37.0,None,Australia,None,", ,",2.079442,0,0,0,0,0,1,0,0,0,0,0,0,1
78867,23,Clayton Christensen,", 67, American business theorist, cancer.",https://en.wikipedia.org/wiki/Clayton_Christensen,31,2020,January,None,None,,cancer,None,None,None,None,None,None,None,None,67.0,None,United States of America,None,None,3.465736,0,0,0,1,1,0,0,0,0,0,0,0,2


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [98]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 926 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [99]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [100]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [101]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "founder of" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [102]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [103]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "ist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [104]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Teachta Dála for Cork East",
    "Fianna Fáil Teachta Dála",
    "congress member",
    "cultural figure",
    "accused of involvement in attempt by Mehmet Ali Ağca to kill Pope John Paul II",
    "son of the last Emperor Bảo Đại",
    "Clerk of Courts for Middlesex County",
    "member of the Birmingham Six",
    "National Farmers' Federation Chief Executive for eight years",
    "son of Emperor Charles I of",
    "Commissioner of Social Security",
    "founder of the Popular Front for the Liberation of",
    "Director of the Office of Management and Budget",
    "last known native speaker of the Eyak language",
    "deputy head of Ingushetia Supreme Court",
    "deputy",
    "pretender to the title Emperor of",
    "founding member of Black Consciousness Movement",
    "who supported democracy",
    "last surviving son of Nelson Mandela",
    "sister of John F Kennedy",
    "confidant to Vladimir Putin",
    "envoy in",
    "Civil Rights pioneer",
    "Prince",
    "member of City Council",
    "embalmer",
    "Mohegan tribal matriarch",
    "Civil Rights Activist",
    "Activist",
    "Fianna Fáil Teachta Dála",
    "NGO founder",
    "Yang di Pertuan Agong",
    "municipal",
    "self proclaimed head of state of the Principality of Seborga micronation",
    "younger son of Ayatollah Ruhollah Khomeini and father of Hassan Khomeini",
    "member of the Senate",
    "interpreter at the Nuremberg Trials",
    "MLA for Pictou West",
    "rd Head of the Imperial Dynasty",
    "member of the Fair Play Committee",
    "opposition figure",
    "Foreign Service Officer",
    "holocaust denier",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "videographer",
    "era cultural functionary",
    "cultural functionary",
    "cultural worker",
    "horticultural",
    "CBS News sound technician",
    "motion picture special effects pioneer",
    "journalism at Columbia University",
    "Executive Editor of for years",
    "head of NBC daytime",
    "founder of Village Roadshow Limited",
    "transgender showgirl",
    "professional transsexual whistler",
    "founder of the Stratford Festival of",
    "Bass with the City Metropolitan Opera",
    "theatrical flying effects specialist",
    "sound effects creator",
    "horror  of the s",
    "Poet Laureate",
    "s on Southern cuisine",
    "TV and",
    "pantomime dame",
    "founder of ITN News at Ten",
    "and scenarist",
    "first female",
    "founder of the San Diego Comic Con",
    "saloon keeper",
    "lion tamer",
    "founder of Pacific News Service",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "bush walker who rediscovered the Takahē in",
    "sumo yokozuna",
    "yokozuna",
    "World Rally champion",
    "Mets fan known for his signs in the crowd",
    "competitive eater",
    "shortstop",
    "MLB All Star and League batting champion",
    "MLB All Star",
    "MLB umpire",
    "Baltimore Orioles",
    "NBA basketball",
    "NBA referee",
    "former National Hockey League and World Hockey Association defenseman",
    "Wimbledon winner",
    "fan of the Denver Broncos",
    "go master",
    "CEO",
    "hillclimbing competitor",
    "founder of the Peach Bowl",
    "molecular genetics",
    "first class wicketkeeper",
    "stadium groundskeeper",
    "Quizzing world champion",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "video game pioneer",
    "electronics innovator",
    "pioneer in cognitive behavioral therapy",
    "developer of Clean Language",
    "pioneer in anaesthetics",
    "specialist in infectious diseases",
    "Nobel Laureate in Physics",
    "creator of the Apple Macintosh",
    "Nobel Laureate in Chemistry",
    "natural",
    "optician and",
    "games developer",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "retail pioneer",
    "founder of Carl Jr restaurants",
    "developer active in City and Aspen",
    "City developer",
    "shopping mall developer",
    "innkeeper",
    "founder of Brookstone",
    "corporate raider",
    "prostitute",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "pioneer of Dark Age archaeology",
    "founder of the Holocaust Memorial Museum",
    "railway preservation pioneer",
    "hispanist",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "Chief of Staff of the IRA and the Official IRA",
    "member of the Provisional IRA",
    "Chief of Staff of the Defense Forces",
    "Chief of Staff of the Defence Forces",
    "Deputy Chief of Staff",
    "first Chief of Staff",
    "representative of the Abraham Lincoln Brigade",
    "FARC second in command",
    "first Defence Force service person killed in",
    "army lieutenant imprisoned by the NKVD and purported escapee",
    "army Major General",
    "army reservist",
    "army medic",
    "army major",
    "army",
    "former head of Sinn Féin at Stormont",
    "member of the Citizens' Commission to Investigate the FBI",
    "former Director of the FBI",
    "who spied for the Union during the Cold War",
    "Chief of Special Operations for the Joint Chiefs of Staff",
    "wife of Lieutenant General Hal Moore",
    "Director General of the Red Cross",
    "Director General of MI",
    "army Major General",
    "Lieutenant General",
    "General",
    "bombardier",
    "Colditz Castle escapee",
    "private first class",
    "Z Special Unit member",
    "patriot",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "ceremonial head of the Nyingma school of Buddhism",
    "Grand Master of the Order of",
    "founder of Transcendental Meditation movement",
    "former primate of the Orthodox Archdiocese of",
    "Anglican co primate",
    "s Anglican primate",
    "antipope self proclaimed Gregory XVII in",
    "zen master",
    "Orthodox Patriarch of Alexandria",
    "first hierarch of the ROCOR",
    "Syro Malabar hierarch",
    "Apostolic hierarch",
    "Maronite hierarch",
    "hierarch",
    "Samaritan High Priest",
    "Head of the Ecclesiastical Mission in of the ROCOR",
    "Lutheran",
    "Grand Mufti",
    "Hierarch in Patriarchate of Constantinople",
    "Orthodox Metropolitan of Florina",
    "Grand Marabout of the Mourides",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "founder of Children World charity",
    "founder of the Vegan Society",
    "animal sanctuary keeper",
    "hippie",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    'bandit known as "Jungle Cat"',
    "bankrobber",
    "last surviving conspirator in the assassination of Mahatma Gandhi",
    "technical mastermind of the and Bali bombings",
    "electrician and mechanic",
    "brothel keeper",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "phone phreak",
    "graduate",
    "Son of Chris Benoit and Nancy Benoit",
    "murder",
    "allegedly killed for refusing to wear hijab",
    "Student at E O Green Junior High School",
    "girl born with two heads",
    "backpacker",
    "college junior",
    "lottery winner",
    "alleged illegitimate daughter of Warren G Harding",
    "shop assistant",
    "savant",
    "municipal clerk",
    "father of Michael Jackson molestation accuser",
    "girl with rare sirenomelia condition",
    "blind mechanic",
    "housekeeper and killer",
    "World Cup oracle octopus",
    "embassy worker",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "grey parrot",
    "hybrid orangutan who learned Sign Language",
    "orangutan",
    "Siberian tiger at San Francisco Zoo",
    "Chief Mouser to the Cabinet Office",
    "Pit Bull Terrier",
    "protected lion",
    "stallion",
    "sea lion",
    "dolphin with a prosthetic tail",
    "dolphin",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [105]:
# Hard-coding cause_of_death for entry with value in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Jacob_Kovco"].index
df.loc[index, "cause_of_death"] = "firearm accident"

# Dropping entry with link that points to event page rather than individual page
index = df[df["link"] == "https://en.wikipedia.org/wiki/Thor_Hesla"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

# Dropping entry with link that points to husband's page rather than individual's page
index = df[df["link"] == "https://en.wikipedia.org/wiki/Don_Geronimo#Family"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [106]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [107]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 58s
Wall time: 1min 58s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
29410,4,Günther Rall,", 91, German Luftwaffe flying ace during World War II, heart attack.",https://en.wikipedia.org/wiki/G%C3%BCnther_Rall,160,2009,October,None,None,,heart attack,None,None,None,None,None,None,None,None,91.0,None,Germany,Germany,None,5.081404,0,0,0,0,0,0,0,1,0,0,0,0,1
22016,11,Pat Corley,", 76, American actor , heart failure.",https://en.wikipedia.org/wiki/Pat_Corley,3,2006,September,,None,,heart failure,None,None,None,None,None,None,None,None,76.0,None,United States of America,None,,1.386294,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [108]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 737 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [109]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [110]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [111]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "Poughkeepsie" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [112]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [113]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "Poughkeepsie"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [114]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "last surviving plaintiff in",
    "head of DFAT",
    "founder of National Grandparents Day",
    "member of Baltimore City Council",
    "Garioch",
    "honorary consul of the to the",
    "commissioner of the Northwest Territories",
    "qadi",
    "islamist",
    "secessionist",
    "member of the Federal Council",
    "Privacy Commissioner",
    "High Commissioner for Western Pacific",
    "AIDS denialist",
    "first government ombudsman",
    "national policy advisor",
    "policy advisor",
    "Indiana Congressman from to",
    '"Mother of the Modern Conservationist Movement"',
    "widow of Henri",
    "member of the Federal Council",
    "member of the Politburo",
    "wife of the pretender to throne",
    "politics",
    "carpenter",
    "volunteer medic",
    "Counsel",
    "Alabama  for the Middle District of Alabama",
    "councillor",
    "of the Peerage of",
    "watcher",
    "emigrant",
    "recluse",
    "pro life",
    "Second Lady",
    "Supreme Leader",
    "Executive",
    "labour organizer",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "sign language interpreter",
    "interior and",
    "vibraphonist",
    "hobbyist",
    "Director of Voice of",
    "founder of magazine",
    "quilter",
    "and creator of Amazing Sea Monkeys",
    "companion",
    "advisor of Miss titleholders for the Miss Universe contest",
    "founding member of The Ramones",
    "member of the ballroom scene",
    "band member of Bee Gees",
    "band member of T Rex",
    "and patron of Jean Cocteau",
    "founder of National Ballet School",
    "mother of Bob Marley",
    "decorative flag pioneer",
    "reggae pioneer",
    "master carpenter",
    "antiquities dealer",
    "nightclub proprietor",
    "Pulitzer prize winning",
    "ukulelist",
    "Chef",
    "Gypsy",
    "viola",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "hammer throw champion",
    "bodybuilding promoter",
    '"Road Warrior Hawk"',
    "time Pacific Rally champion",
    "member of st Little League World Series championship team",
    "member of the Vanderbilt family",
    "bodybuilding pioneer",
    "member of st Little League World Series championship team",
    "professional gambler",
    "climbing and caving pioneer",
    "adventurist",
    "sidecarcross rider and the first ever Sidecarcross World Championship",
    "enduro rider",
    "Thoroughbred breeder",
    "founder of Goju Ryu Seigokan Karatedo",
    "founder of Paris Saint Germain F C",
    "Paralympian",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "born geriatrician",
    "geriatrician",
    "developer of ventricular assist device",
    "agriculture",
    "meteorology",
    "invertebrate zoology",
    "zoology",
    "early member of Mensa",
    "Instruments founder",
    "geometer",
    "creator of the Leslie speaker",
    "aircraft developer",
    "dietician",
    "Associate Director of the",
    "fitness professional",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "founder of Rockmount Ranch Wear",
    "founder of Benihana restaurants",
    "founder of Movado Group",
    "founder of Monster com",
    "crude oil marketer",
    "direct marketer",
    "marketer",
    "member of the Rockefeller family",
    "valuation surveyor",
    "and Gold Gym founder",
    "founder of McDonald",
    "carpet specialist",
    "visual merchandiser",
    "investment professional",
    "founder of the Auto Train Corporation",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "ASL interpreter",
    "Pursuivant",
    "Faculty Emeritus",
    "first Chancellor of the University of",
    "scolar",
    "Slavist",
    "college principal",
    "palaeo",
    "founder of the Canoe Museum",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "al Qaeda second in command",
    "al Qaeda chemical",
    "member of the INLA",
    "head of the Defence Force",
    "winner of the Albert Medal",
    "SS escort of Adolf Hitler",
    "fourth head of the",
    "crew member of the bombing of Hiroshima during the",
    "member of the Liberation Tigers of Eelam",
    "member of Liberation Tigers of Eelam",
    "member of the INLA",
    "member of al Qaeda",
    "member of the SS",
    'first Naval Flight Officer to achieve the "Flag" rank',
    "and Women Royal Naval Service personnel",
    "volunteer of the UVF",
    "resistance movement member",
    "frogman",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "Anglican Primate",
    "muslim sufi",
    "muslim",
    "papal",
    "member of this body since its beginnings",
    "shaman and witch",
    "Sufist",
    "liturgical",
    "second rebbe of",
    "Orthodox Metropolitan of Cluj Napoca",
    "Orthodox metropolitan",
    "Orthodox",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "crisis support counsellor",
    "mother of Diana",
    "welfare worker",
    "charity patron",
    "housewife",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "moonshiner",
    "perjurer against Jomo tta",
    "escort agency proprietor",
    "kidnapper and rapist",
    "kidnapper",
    "Mafioso",
    "gunman involved in Mumbai attacks",
    "mafiosi",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "subject of iconic photo of internment of s",
    "Congressional intern",
    "internee",
    "last certified Civil War widow",
    "patient in right to die case",
    "bird feeder",
    "mother of Joseph Estrada",
    "half sister of Venus and Serena Williams",
    "electrician",
    "homosexual",
    "baby",
    "vagrant",
    "adoptee",
    "arrestant",
    "Poughkeepsie",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "Standardbred colt",
    "Major Mitchell cockatoo",
    "scaleless carp",
    "common carp",
    "chihuahua",
    "pond cypress tree",
    "noted for his upright stature and humanlike traits",
    "Vizsla",
    "National Park gray wolf",
    "bonobo involved in language studies",
    "body modifier",
    "Pinta Island tortoise endling",
    "Crested Dog",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [115]:
# Dropping entry with link that points to historical page rather than individual page
index = df[df["link"] == "https://en.wikipedia.org/wiki/Bruce_Trevorrow"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

# Dropping entry with link that points to group page rather than individual page
index = df[
    df["link"] == "https://en.wikipedia.org/wiki/Madelyn_and_Stanley_Dunham"
].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

# Dropping entry with link that points to group page rather than individual page
index = df[
    df["link"]
    == "https://en.wikipedia.org/wiki/American_Civil_War_widows_who_survived_into_the_21st_century#Alberta_Martin_(1906–2004)"
].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [116]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [117]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 33s
Wall time: 1min 33s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
13938,13,Beatrice Macola,", 36, Italian actress, cerebral infarction.",https://en.wikipedia.org/wiki/Beatrice_Macola,3,2001,December,None,None,,cerebral infarction,None,None,None,None,None,None,None,None,36.0,None,Italy,None,None,1.386294,0,0,0,0,0,1,0,0,0,0,0,0,1
26071,24,Eddie Davidson,", 35, American convicted spammer and prison escapee, suicide by gunshot.",https://en.wikipedia.org/wiki/Eddie_Davidson,4,2008,July,None,None,,suicide by gunshot,None,None,None,None,None,None,None,None,35.0,None,United States of America,None,None,1.609438,0,0,0,0,0,0,0,0,0,1,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [118]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 587 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [119]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [120]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [121]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "surveyor" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [122]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [123]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "surveyor"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [124]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Knesset member",
    "anti Zionist",
    "Emperor in exile and son of Haile Selassie I",
    "Commissioner of Elections",
    "th Duke of",
    "Zionist",
    "member of the",
    "member of",
    "th generation descendant of Confucius",
    "Royal",
    "clansman",
    "Arabian sheik",
    "Congressman",
    "councilman",
    "national security advisor",
    "pretender to the throne of the dom of Araucania and Patagonia",
    "titular Maharaja of Travancore",
    "to pass the Maryland bar exam",
    "Senior Economist",
    "calculator",
    "independence pioneer",
    "superintendent",
    "school board member",
    "politic",
    "eugenicist",
    "of jurisprudence",
    "last imperial eunuch",
    "collaborator",
    "civic",
    "associate of Carlos Menem",
    "gay and transgender youth",
    "junker",
    "emperor of the Nguyễn dynasty",
    "propagandist",
    "Reformist",
    "queen",
    "advisor to Chancellor Konrad Adenauer",
    "Virgin Islander  of the Virgin Islands",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "widow of Superman co creator Jerry Siegel",
    "founder of",
    "hula master",
    "website",
    "creator of",
    "cabarettist",
    "star",
    "independent",
    "AV idol",
    "debutante",
    "pop",
    "Broadway",
    "accomplished  from",
    "female impersonator",
    "epilepsy",
    "draughtsman and",
    "draughtsman",
    "and shooting continuity specialist",
    "kabarettist",
    "festival organiser",
    "punk",
    "Freemasonry grand master",
    "maestro and Tar master",
    "fantasy",
    "drag king",
    "drag",
    "fairy tale narrator",
    "gardening",
    "carnival king",
    "Kathakali maestro",
    "Artist",
    "coin dealer",
    "adman",
    "swordsmith",
    "acrobat",
    "Cajun",
    "hypnotist",
    "horticulture",
    "Mridanga maestro",
    "showgirl",
    "housemaid and seamstress in the",
    "and taxidermist",
    "Eurythmist",
    "of short stories",
    "and choral works",
    "photorealist who created large scale still lifes",
    "Artist",
    "Art",
    "Actress",
    "for MGM",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "masters athletics champion",
    "college basketball",
    "basketballer",
    "Paralympic competitor",
    "referee of the FIFA World Cup final",
    "athletics promoter",
    "twice cc MotoGP Champion",
    "Bundesliga referee",
    "Formula One mechanic",
    "Formula One",
    "quadruple world champion",
    "snowmobile competitor",
    "diving",
    "Athletics competitor",
    "linebacker for the University of Hurricanes",
    "National Basketball Association referee",
    "gridiron",
    "Alpinist",
    "flyweight bpxer",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "Internet pioneer",
    "botany",
    "Internet",
    "in vitro fertilisation pioneer",
    "cryonicist",
    "beekeeper",
    "rosarian",
    "NASA mission design specialist",
    "programming",
    "gerontology",
    "pioneer in the development of digital sound",
    "plant breeder",
    "thermodynamics and cryogenics",
    "flight mechanic",
    "mechanic",
    "ecology pioneer",
    "topographer",
    "IT specialist",
    "physcisist and pioneer in the development of radar",
    "pioneer in gastroscopy and endoscopy",
    "automation pioneer",
    "pychiatrist",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "corporate",
    "petroleum developer",
    "community developer",
    "mall developer",
    "developer",
    "Industrialist",
    "chocolatier",
    "product marketeer",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "school founder",
    "compiler of the crossword since",
    "crossword puzzle constructor",
    "crossword",
    "hereditary knight",
    "history",
    "anthroposophist",
    "epigraphist",
    "archive pioneer",
    "school principal",
    "school founder",
    "Byzantinist",
    "Map Curator at the Museum",
    "Ojibwe traditionalist",
    "labourer",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "exile",
    "seventh son of Muammar Gaddafi",
    "lifeboatman",
    "last surviving ANZAC at the Gallipoli campaign during World War I",
    "Director of the Women",
    "Major",
    "escapee",
    "Navajo Code Talker",
    "private detective",
    "commando",
    "highway patrolman",
    "aquanaut",
    "sniper during WWII",
    "and guerilla",
    "guerilla",
    "World War I hero",
    "rescuer of Jews",
    "Ryadovoy and Hero of the Union for his actions during WWII",
    "wife of Oskar Schindler who helped save the lives of",
    "POW escaper",
    "land surveyor",
    "surveyor",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "New Age",
    "Latter day Saints",
    "Hasidic",
    "Baptist",
    "friar",
    "creationist",
    "presbyter",
    "Religious Sister",
    "Metropolitan",
    "Rōshi",
    "Baháʼí Faith convert",
    "Archdeacon of",
    "Buddhism",
    "Maronite Patriarch of Antioch and the Whole Levant",
    "Friar Minor",
    "Father",
    "Grand Imam of al Azhar",
    "Calvinist",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "volunteer",
    "counsellor",
    "rabbanit",
    "community development worker",
    "development worker",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "juvenile offender",
    "mafioso",
    "airplane hijacker",
    "underworld figure",
    "train thief",
    "tax evader",
    "gang member",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "tourist",
    "domestic helper",
    "charity worker",
    "Romanov impostor",
    "imposter",
    "shoeshiner",
    "speed knitter",
    "immigrant to the",
    "immigrant to",
    "immigrant",
    "prodigy mental",
    "schoolgirl",
    "First person born on South",
    "Mother of Bruce Lee",
    "tall man",
    "pretender who claimed connection to various thrones",
    "man and unidentified body and missing person case",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "Thoroughbred",
    "cross eyed opossum at Leipzig Zoo",
    "koala made famous after the Black Saturday bushfires",
    "the world first cloned mammal",
    "giant sequoia tree",
    "plane tree",
    "loblolly pine",
    "greater flamingo",
    "zoo tapir",
    "wild turkey",
    "Greyhound",
    "walrus",
    "beagle",
    "Terrier",
    "golden retriever",
    "last living triple crown winner on th anniversary of winning Derby",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [125]:
# Hard-coding cause_of_death for entry with value in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Michael_Jeter"].index
df.loc[index, "cause_of_death"] = "epileptic seizure"

# Hard-coding info_2 to correctly categorize entry
index = df[
    df["link"]
    == "https://en.wikipedia.org/wiki/John_Bernard_Philip_Humbert,_9th_Count_de_Salis-Soglio"
].index
df.loc[index, "info_2"] = "count Major"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [126]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [127]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 58s
Wall time: 1min 58s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
2960,21,Rudy Perpich,", 67, American politician and Governor of Minnesota, colorectal cancer.",https://en.wikipedia.org/wiki/Rudy_Perpich,6,1995,September,None,None,,colorectal cancer,None,None,None,None,None,None,None,None,67.0,None,United States of America,None,None,1.945910,0,0,0,0,0,0,0,0,1,0,0,0,1
68916,8,Peter Fehlner,", 86, American Roman Catholic priest and theologian.",https://en.wikipedia.org/wiki/Peter_Fehlner,17,2018,May,None,None,,None,None,None,None,None,None,None,None,None,86.0,None,United States of America,Italy,None,2.890372,0,0,1,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [128]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 420 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [129]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [130]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [131]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "Salvation" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [132]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [133]:
# Example code to quick-check a specific entry
df[df["info_2"] == "Salvation"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [134]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "Civil Rights Movement",
    "monument",
    "prefect",
    "community",
    "Gandhian",
    "First Female Barrister in",
    "Constitutional",
    "government advisor",
    "polician",
    "stenographer",
    "Lakota Chief",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "barman",
    "comedy",
    "scenographer",
    "visual effects supervisor",
    "feuilletonist",
    "candidate",
    "undertaker",
    "abstract sculpor",
    "abstract",
    "bugler",
    "ukuleleist",
    "soloist",
    "antiquarian",
    "festival founder",
    "Sephardic",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "junior ice dancing champion",
    "cheerleading innovator",
    "motor cycle champion",
    "race",
    "ironman",
    "ovarian cancer",
    "matador",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "psychoanalysis",
    "psychological",
    "microscopist",
    "aquarist",
    "shipwright",
    "electronics",
    "plant systematist",
    "antenna specialist",
    "bioacoustics specialist",
    "thermodynamicist",
    "cybersecurity",
    "archaeometrist",
    "prosthodontist",
    "electrotechnician",
    "cybernetician",
    "zootechnician",
    "robotician",
    "astronomy",
    "nature",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "bathroom furnishings",
    "real estate broker",
    "real estate",
    "tea company founder",
    "supermarket founder",
    "motor industry",
    "insurance advisor",
    "mortgage lender",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "school",
    "Chancellor of York University",
    "Hebraist",
    "celtist",
    "literature",
    "Mongolist",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "al Qaeda member",
    "al Qaeda member in custody",
    "IS recruiter",
    "brigadier",
    "RAF air observer during the Battle of",
    "Ranger",
    "Maroon Beret",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "pagan",
    "motivational",
    "Unitarian",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "Batman impersonator",
    "creator",
    "organisation founder",
    "servant",
    "Rotarian",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["procurer", "heroin dealer", "cannabis smuggler"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "heaviest man",
    "podiatrist",
    "retiree",
    "killer",
    "infant",
    "supercentarian",
    "beautician",
    "tattooed man",
    "dental assistant",
    "transgender girl",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "zoo cheetah",
    "manatee",
    "giant pouched rat",
    "albino buffalo",
    "tigress",
    "trapdoor spider",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [135]:
# Hard-coding cause_of_death for entry with value in info_2
index = df[
    df["link"] == "https://en.wikipedia.org/wiki/Noel_Johnson_(basketball)"
].index
df.loc[index, "cause_of_death"] == "ovarian cancer"

82151    False
Name: cause_of_death, dtype: bool

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [136]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [137]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 50.2 s
Wall time: 50.2 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
69320,4,C. M. Newton,", 88, American Hall of Fame college basketball coach .",https://en.wikipedia.org/wiki/C._M._Newton,17,2018,June,"Alabama Crimson Tide, Vanderbilt Commodores and administrator Wildcats",None,,None,None,None,None,None,None,None,None,None,88.0,None,United States of America,None,"Alabama Crimson Tide, Vanderbilt Commodores and administrator Kentucky Wildcats",2.890372,0,0,0,0,0,0,1,0,0,0,0,0,1
1955,16,Albert Hackett,", 95, American dramatist and screenwriter.",https://en.wikipedia.org/wiki/Albert_Hackett,5,1995,March,None,None,,None,None,None,None,None,None,None,None,None,95.0,None,United States of America,None,None,1.791759,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [138]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 350 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [139]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [140]:
# Code to check each value
roles_list.pop()

''

<IPython.core.display.Javascript object>

In [141]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "negotiator" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [142]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

[]

<IPython.core.display.Javascript object>

In [143]:
# Example code to quick-check a specific entry
df[df["info_2"] == "negotiator"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [144]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "advisor",
    "reproductive rights",
    "interpreter",
    "count",
    "Fascist",
    "transgender pioneer",
    "idealist",
    "oberst",
    "reality show",
    "social democrat",
    "Public Service",
    "and mistress of John F Kennedy",
    "and wife of John F Kennedy Jr",
    "Commissioner",
    "Domestic Policy Council and",
    "third  of",
    "Chief Ombudsman",
    "Revisionist",
    "far right",
    "anti globalization demonstrator",
    "wife of former chancellor Helmut Kohl",
    "wife of Herzog",
    "Empress of",
    "AIDS policy coordinator",
    "disability",
    "syndicalist",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "tresero",
    "narrator",
    "culturist",
    "doorman",
    "master",
    "sword swallower",
    "design",
    "gaffer",
    "walking guide",
    "rose",
    "marimbist",
    "juggler",
    "Lovelock Paiute",
    "events promoter",
    "club promoter",
    "winner",
    "and Menards spokesman",
    "cantaora and bailaora",
    "design",
    "fly tyer",
    "raconteur",
    "Kantor",
    "Chief Engraver of the Mint",
    "drum and bass MC",
    "grime MC",
    "MC",
    "swing and",
    "ska",
    "Staple Singers",
    "and son of Carl Hoffmann",
    "mustachioed",
    "Journalist",
    "swing",
    "qawwali",
    "B movie",
    "reciter",
    "events promoter",
    "sonneur",
    "folk and Cel",
    "pannist",
    "cutler",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "footbaler",
    "professional",
    "gambler",
    "roller derby promoter",
    "college athletics commissioner",
    "sport",
    "ski and snowboarding",
    "curling champion",
    "racing",
    "ski",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "psycho",
    "perma",
    "pioneer in the field of industrial",
    "computational",
    "fruit breeder",
    "technical",
    "cardiology",
    "gunsmith",
    "anesthetist",
    "ecological",
    "industrial",
    "hydrodynamicist",
    "biological",
    "birder",
    "surgery",
    "Mechanical Engineering",
    "paediatrics",
    "electrical",
    "Ayurveda practitioner",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "goat breeder",
    "s breeder",
    "automobile dealer",
    "crofter",
    "wine merchant",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "tourism promoter",
    "first Director of the National Scientific Documentation Centre",
    "Philanthropist",
    "Latinist",
    "seanchaí and the last known monolingual speaker",
    "polyglot",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "Army officer",
    "sniper",
    "ace",
    "Stasi member",
    "Army",
    "officer",
    "yeoman",
    "Commandant of Coast Guard",
    "Lieutenant Colonel",
    "staff member during the War",
    "security",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "Mormo",
    "Salvationist",
    "Islam",
    "ascetic",
    "spiritual",
    "wiccan",
    "patriarch of The",
    "mother of Donald Trump",
    "self proclaimed prophet",
    "Latter Day Saint movement",
    "deacon",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = ["social worke", "social work", "charity founder"]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "gunman",
    "heroin smuggler",
    "serial rapist",
    "impostor",
    "conspirator in the Watergate scandal",
    "Mafia",
    "crime lord",
    "embezzler",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "courier",
    "émigré to City",
    "girl",
    "unnamed motel guest",
    "quintuplet",
    "plagiarist",
    "Romani citizen",
    "housemaid",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = ["comedic goat", "red kangaroo", "gelding"]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [145]:
# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Diethard_Hellmann"].index
df.loc[index, "info_2"] = "spiritual Kantor"

# Hard-coding cause_of_death for entry with value in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Paul_Koslo"].index
df.loc[index, "cause_of_death"] = "pancreatic cancer"

# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Aziz_Mian"].index
df.loc[index, "info_2"] = "spiritual qawwali"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [146]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [147]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 14s
Wall time: 1min 14s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
57836,11,Richard Ransom,", 96, American businessman .",https://en.wikipedia.org/wiki/Richard_Ransom,5,2016,April,Hickory Farms,None,,None,None,None,None,None,None,None,None,None,96.0,None,United States of America,None,Hickory Farms,1.791759,0,0,0,0,1,0,0,0,0,0,0,0,1
85499,24,Harry Ryan,", 63, Irish hurler .",https://en.wikipedia.org/wiki/Harry_Ryan_(hurler),3,2020,November,Kilkenny,None,,None,None,None,None,None,None,None,None,None,63.0,None,Ireland,None,Kilkenny,1.386294,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [148]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 141 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [149]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [150]:
# Code to check each value
roles_list.pop()

''

<IPython.core.display.Javascript object>

In [151]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "Union" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [152]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

[]

<IPython.core.display.Javascript object>

In [153]:
# Example code to quick-check a specific entry
df[df["info_2"] == "Union"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [154]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "coroner",
    "consort",
    "Afghani",
    "citizen",
    "social and feminism",
    "arbitrator",
    "taxation",
    "promoter",
    "king",
    "social policy",
    "clerk",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "stage",
    "theater and",
    "plumber",
    "pimp",
    "humourist",
    "rakugoka",
    "folk",
    "vexillographer",
    "actor",
    "pancreatic cancer",
    "sommelier",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = ["rikishi", "Stampede Wrestling", "inter"]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = ["ayurvedic practitioner", "ampelographer", "agri", "pioneer", "breeder"]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = ["automobile"]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = ["Africanist", "pedagogist"]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "homicide detective",
    "guerrillero",
    "Righteous Among the Nations",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = ["tulku", "marja'", "patriarch"]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["consigliere", "jogger"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = ["quadriplegic and vegetative state right to die figure"]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "Border Collie with the largest tested non human memory",
    "sheep",
    "left coiled snail",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [155]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [156]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 23 s
Wall time: 23 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
80977,17,Gene Shay,", 85, American disc jockey , co-founder of Philadelphia Folk Festival, COVID-19.",https://en.wikipedia.org/wiki/Gene_Shay,4,2020,April,WXPN,None,,co founder of Philadelphia Folk Festival,COVID,None,None,None,None,None,None,None,85.0,None,United States of America,None,WXPN,1.609438,0,0,0,0,0,1,0,0,0,0,0,0,1
97975,3,Saafi Boulbaba,", 36, Tunisian footballer , traffic collision.",https://en.wikipedia.org/wiki/Saafi_Boulbaba,4,2022,June,"Espérance Sportive de Tunis, A S Kasserine, MC El Eulma",None,,traffic collision,None,None,None,None,None,None,None,None,36.0,None,Tunisia,None,"Espérance Sportive de Tunis, A S Kasserine, MC El Eulma",1.609438,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [157]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 101 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [158]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [159]:
# Code to check each value
roles_list.pop()

''

<IPython.core.display.Javascript object>

In [160]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "locksmith" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [161]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

['locksmith']

<IPython.core.display.Javascript object>

In [162]:
# Example code to quick-check a specific entry
df[df["info_2"] == "W"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [163]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = ["sex", "autism", "activist", "IT worker"]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "gallerist",
    "southern",
    "string",
    "clothing stylist and merchandiser",
    "stonemason",
    "Wagner family member",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = ["Twitch streamer", "tai chi practitioner", "equestrienne"]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = ["copist", "roboteer", "locksmith"]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "comprador",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "spectros",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "resistance movement rescuer",
    "resistance",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "astrology",
    "Baháʼí",
    "esotericist",
    "Novominsk rebbe",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = ["considered the first person cured of HIV AIDS"]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = ["Powerball", "dressing room attendant", "extended coma recoverer"]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = ["alligator"]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [164]:
# Hard-coding info_2 value for entry to correctly categorize
index = df[df["link"] == "https://en.wikipedia.org/wiki/Manolis_Glezos"].index
df.loc[index, "info_2"] = "resistance activist"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [165]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [166]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 15.6 s
Wall time: 15.6 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
91842,6,Oleg Shapovalov,", 58, Ukrainian politician, president of the Kharkiv Oblast Council .",https://en.wikipedia.org/wiki/Oleg_Shapovalov,3,2021,August,,None,,president of the Kharkiv Oblast Council,None,None,None,None,None,None,None,None,58.0,None,Ukraine,None,2005 2006,1.386294,0,0,0,0,0,0,0,0,1,0,0,0,1
982,20,"Roy Simmons, Sr.",", 92, American lacrosse coach.",https://en.wikipedia.org/wiki/Roy_Simmons_Sr.,20,1994,August,None,None,,None,None,None,None,None,None,None,None,None,92.0,None,United States of America,None,None,3.044522,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [167]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 83 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [168]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [169]:
# Code to check each value
roles_list.pop()

''

<IPython.core.display.Javascript object>

In [170]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "counsel" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [171]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

['counsel']

<IPython.core.display.Javascript object>

In [172]:
# Example code to quick-check a specific entry
df[df["info_2"] == "counsel"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
95556,23,Edgar S. Cahn,", 86, American law professor and counsel.",https://en.wikipedia.org/wiki/Edgar_S._Cahn,19,2022,January,None,None,counsel,None,None,None,None,None,None,None,None,None,86.0,None,United States of America,None,None,2.995732,0,0,0,1,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [173]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = ["rioter", "counsel"]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "ghazal",
    "pizzaiolo",
    "paparazzo",
    "classical and",
    "special effects supervisor",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = ["Tang Soo Do practitioner", "long distance bikepacker"]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = ["homeopath", "IT"]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = ["Sanskrit"]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = ["anti ETA Civil Guard"]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = ["mufti", "theological"]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["bank", "Satanist"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = []
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = ["eagle", "brown kiwi", "alpaca"]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [174]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [175]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 11.7 s
Wall time: 12.1 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
9919,28,Imre Zichy,", 90, Hungarian count and tennis player.",https://en.wikipedia.org/wiki/Imre_Zichy,11,1999,September,None,None,,None,None,None,None,None,None,None,None,None,90.0,None,Hungary,None,None,2.484907,0,0,0,0,0,0,1,0,1,0,0,0,2
68157,19,David Bischoff,", 66, American novelist and television writer .",https://en.wikipedia.org/wiki/David_Bischoff,6,2018,March,,None,,None,None,None,None,None,None,None,None,None,66.0,None,United States of America,None,,1.945910,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [176]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 72 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [177]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [178]:
# Code to check each value
roles_list.pop()

''

<IPython.core.display.Javascript object>

In [179]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "aw" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [180]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

[]

<IPython.core.display.Javascript object>

In [181]:
# Example code to quick-check a specific entry
df[df["info_2"] == "aw"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [182]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = ["seal inspector"]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "etcher",
    "gastronomic",
    "stylist",
    "pantomimer",
    "cheesemonger",
    "baker",
    "influencer",
    "animation",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = ["outdoor"]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = ["immunoligist"]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = ["language", "germanist"]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = []
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = ["hijacker"]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = ["groupie", "vandweller"]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [183]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [184]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 8.59 s
Wall time: 8.64 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
55476,2,Edward Soja,", 75, American geographer.",https://en.wikipedia.org/wiki/Edward_Soja,18,2015,November,None,None,,None,None,None,None,None,None,None,None,None,75.0,None,United States of America,None,None,2.944439,0,0,0,1,0,0,0,0,0,0,0,0,1
49815,16,Mary Speer,", 89, American southern gospel singer .",https://en.wikipedia.org/wiki/Mary_Speer,8,2014,September,Speer Family,None,,None,None,None,None,None,None,None,None,None,89.0,None,United States of America,None,Speer Family,2.197225,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [185]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 61 entries without any known_for category.


<IPython.core.display.Javascript object>

In [186]:
# Checking value counts for info_2
df["info_2"].value_counts()

       97983
and       18
of         3
s          2
n          1
l          1
ing        1
in         1
era        1
ian        1
Name: info_2, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- The remaining values in info_2 are extraneous and we can now drop the column.
- We have fewer than 100 entries without any `known_for` categorization remaining.  We will search for them in the other `info_` columns.

#### Dropping `info_2`

In [187]:
# Dropping info_2
df.drop("info_2", axis=1, inplace=True)

<IPython.core.display.Javascript object>

#### Observations:
- Prior to searching the other columns for remaining `known_for` values, we should export our dataframe and start a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean13.db]()

In [189]:
# Exporting dataframe

# Saving dataset in a SQLite database
conn = sql.connect("wp_life_expect_clean13.db")
df.to_sql("wp_life_expect_clean13", conn, index=False)

# Chime notification when cell executes
chime.success()

<IPython.core.display.Javascript object>

# [Proceed to Data Cleaning Part 14]()